# DB 적재용 더미 데이터 생성

In [1]:
from faker import Faker
from sdv.metadata import MultiTableMetadata
from sdv.metadata import Metadata
from sdv.single_table import GaussianCopulaSynthesizer

import os
import re
import random
from datetime import timedelta, datetime, timezone
from collections import defaultdict, Counter
import uuid

import hashlib
import os
import base64

import numpy as np
import pandas as pd

import dask
from dask import delayed
import time
from tqdm import tqdm
import threading

import bcrypt

## 날짜 형식 표준화 (ISO 8601)

이 노트북에서 생성되는 모든 날짜 데이터는 **ISO 8601** 표준 형식을 사용합니다.

- **형식**: `YYYY-MM-DDTHH:MM:SSZ`
- **예시**: `2024-07-01T10:00:00Z`
- **장점**:
  - 국제적으로 표준화된 형식
  - 시간대 정보 포함 (Z = UTC)
  - 다양한 시스템 간 호환성 보장
  - 정렬 및 비교가 용이

In [2]:
# plt.rcParams['font.family'] = 'Nanum Gothic'
# plt.rcParams['axes.unicode_minus'] = False

pd.set_option('display.max_seq_items', None)
# row 생략 없이 출력
pd.set_option('display.max_columns', None)

## 1. 고객 패스워드 복호화

In [3]:
def make_password(password, cost_factor=12):
    """
    bcrypt를 사용하여 암호 해시를 생성합니다.

    • password: 평문 문자열 (UTF-8)
    • cost_factor: bcrypt의 작업량 계수 (cost factor). 
                   숫자가 클수록 더 많은 계산이 필요하여 안전하지만 느려집니다. 
                   일반적으로 12-14가 권장됩니다.
    """
    # 비밀번호를 UTF-8 바이트로 인코딩합니다.
    password_bytes = password.encode('utf-8')
    
    # 지정된 cost_factor로 솔트(salt)를 생성합니다.
    salt = bcrypt.gensalt(rounds=cost_factor)
    
    # 비밀번호와 솔트를 해싱합니다.
    hash_bytes = bcrypt.hashpw(password_bytes, salt)
    
    # 최종 해시 문자열을 반환합니다 (UTF-8로 디코딩).
    # 이 문자열 안에는 알고리즘, cost factor, salt, 해시 값이 모두 포함됩니다.
    return hash_bytes.decode('utf-8')

## 2. 참조 필요 없는 데이터 생성

In [4]:
# ─── 상수 정의 ───
NUM_USERS = 30000  # 사용자 수를 30,000명으로 조정

# 서비스 기간 설정: 2024.07.01 ~ 2025.07.21 (1년간)
SERVICE_START_DATE = datetime(2024, 7, 1)
SERVICE_END_DATE = datetime(2025, 7, 21)

print(f"🎯 서비스 시나리오:")
print(f"   - 서비스 시작일: {SERVICE_START_DATE.strftime('%Y.%m.%d')}")
print(f"   - 서비스 현재일: {SERVICE_END_DATE.strftime('%Y.%m.%d')}")
print(f"   - 서비스 운영 기간: {(SERVICE_END_DATE - SERVICE_START_DATE).days}일")
print(f"   - 총 사용자 수: {NUM_USERS:,}명")

fake = Faker('ko_KR')

🎯 서비스 시나리오:
   - 서비스 시작일: 2024.07.01
   - 서비스 현재일: 2025.07.21
   - 서비스 운영 기간: 385일
   - 총 사용자 수: 30,000명


### 2-1. Recipes

In [5]:
recipes_24_df = pd.read_csv("data/TB_RECIPE_SEARCH_241226.csv", encoding='utf-8-sig')
print(f"데이터 형태: {recipes_24_df.shape}")
print(f"컬럼명: {list(recipes_24_df.columns)}")
print("\n레시피 제목 샘플 (한글 정상 표시):")
print(recipes_24_df['RCP_TTL'].head(10).to_list())
print("\n요리 종류 샘플:")

데이터 형태: (23192, 19)
컬럼명: ['RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM', 'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN', 'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT', 'RCP_IMG_URL']

레시피 제목 샘플 (한글 정상 표시):
['멸치육수 소고기 떡국 만드는법', '#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육', '우거지감자탕 뼈해장국 끓이는법', '만두전골 레시피 백종원 만두 전골요리 뜨끈하고 진한 국물이 일품', '새해 통삼겹살 무수분 보쌈 삶는법 백종원 보쌈 마늘소스 만들기', '#손두부요리 #두부짜박이만들기 #속초맛집따라하기 #가마솥손두부두부짜박이 #들기름', '돼지 등갈비 바베큐 폭립 만드는법 돼지 등갈비구이', '참치 카나페 만들기 카나페 핑거푸드 만드는 법 참치마요 카나페', '시금치무침 만드는 법 레시피 시금치나물무침 ', '초간단 계란국 끓이는법 팽이버섯 계란국']

요리 종류 샘플:


In [6]:
recipes_23_df = pd.read_csv("data/TB_RECIPE_SEARCH-20231130.csv", 
                        encoding='cp949', 
                        encoding_errors='ignore')

print("✅ 한글 깨짐 문제 해결 완료!")
print(f"데이터 형태: {recipes_23_df.shape}")
print(f"컬럼명: {list(recipes_23_df.columns)}")
print("\n레시피 제목 샘플 (한글 정상 표시):")
print(recipes_23_df['RCP_TTL'].head(10).to_list())
print("\n요리 종류 샘플:")

✅ 한글 깨짐 문제 해결 완료!
데이터 형태: (184991, 18)
컬럼명: ['RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM', 'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN', 'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT']

레시피 제목 샘플 (한글 정상 표시):
['어묵김말이', '두부에 꼬리가 달렸어요!! ', '입안에서 톡톡톡 ', '★현미호두죽', '부들부들 보들보들 북어갈비♥', '토마토스파게티♥', '표고버섯탕수', '달콤한 마늘향이 가득해~ 갈릭치킨♥', '바삭바삭 양파링 튀김 만들기', '참치 삼각김밥 삼총사']

요리 종류 샘플:


In [7]:
def process_recipe_data(recipes_24_df=None, recipes_23_df=None, user_data=None):
    """
    레시피 데이터를 처리하는 함수 (올바른 날짜 로직 적용)
    - 2024년과 2023년 레시피 데이터를 읽어옴
    - 데이터를 결합하고 컬럼명을 변경
    - 사용자별 고유 ID를 생성
    - ✅ 중요: 사용자 가입일 이후에 레시피 등록일을 설정 (논리적 순서 준수)
    
    Args:
        recipes_24_df (pd.DataFrame, optional): 2024년 레시피 데이터
        recipes_23_df (pd.DataFrame, optional): 2023년 레시피 데이터
        user_data (pd.DataFrame, optional): 사용자 데이터 (가입일 참조용)
    
    Returns:
        pd.DataFrame: 처리된 레시피 데이터프레임
    """
    
    # 2024년 레시피 데이터 로드
    if recipes_24_df is None:
        print("📖 2024년 레시피 데이터 로드 중...")
        recipes_24_df = pd.read_csv("data/TB_RECIPE_SEARCH_241226.csv", encoding='utf-8-sig')
        print(f"2024년 레시피 개수: {len(recipes_24_df):,}개")
    else:
        print("♻️  기존 2024년 레시피 데이터 재사용")
        print(f"2024년 레시피 개수: {len(recipes_24_df):,}개")
    
    # 2023년 레시피 데이터 로드
    if recipes_23_df is None:
        print("\n📖 2023년 레시피 데이터 로드 중...")
        recipes_23_df = pd.read_csv("data/TB_RECIPE_SEARCH-20231130.csv", 
                                encoding='cp949', 
                                encoding_errors='ignore')
        print(f"2023년 레시피 개수: {len(recipes_23_df):,}개")
    else:
        print("\n♻️  기존 2023년 레시피 데이터 재사용")
        print(f"2023년 레시피 개수: {len(recipes_23_df):,}개")
    
    # 데이터 결합
    print("\n🔗 데이터 결합 중...")
    total_recipes = pd.concat([recipes_24_df, recipes_23_df], ignore_index=True)
    print(f"전체 레시피 개수: {len(total_recipes):,}개")
    
    # 컬럼명 변경
    print("\n🏷️  컬럼명 변경 중...")
    total_recipes_renamed = total_recipes.rename(columns={
        "RCP_SNO": "id", 
        "RCP_TTL": "title", 
        "CKG_NM": "name", 
        "RGTR_ID": "user_name",
        "RGTR_NM": "user_nickname",
        "INQ_CNT": "view_cnt", 
        "RCMM_CNT": "recommend_cnt", 
        "SRAP_CNT": "scrap_cnt",
        "CKG_MTH_ACTO_NM": "method_type", 
        "CKG_STA_ACTO_NM": "situation_type", 
        "CKG_MTRL_ACTO_NM": "ingredient_type", 
        "CKG_KND_ACTO_NM": "dish_type",
        "CKG_IPDC": "content", 
        "CKG_MTRL_CN": "ingredient_list", 
        "CKG_INBUN_NM": "serving",
        "CKG_DODF_NM": "difficulty", 
        "CKG_TIME_NM": "cooking_time", 
        "FIRST_REG_DT": "original_created_at",  # 원본 날짜는 따로 보관
        "RCP_IMG_URL": "image_url"
    })
    print("✅ 컬럼명 변경 완료")
    
    # 사용자별 고유 ID 생성
    print("\n👤 사용자별 고유 ID 생성 중...")
    user_name_col = 'user_name'
    print(f"사용자 이름 컬럼: {user_name_col}")
    print(f"고유 사용자 수: {total_recipes_renamed[user_name_col].nunique():,}명")
    
    # 고유한 user_id 생성
    unique_users = total_recipes_renamed[user_name_col].unique()
    user_mapping = {user: idx + 1 for idx, user in enumerate(unique_users)}
    total_recipes_renamed['user_id'] = total_recipes_renamed[user_name_col].map(user_mapping)
    
    print(f"✅ 생성된 user_id 범위: 1 ~ {total_recipes_renamed['user_id'].max()}")
    
    # 🔧 핵심 개선: 올바른 날짜 로직 (사용자 가입일 → 레시피 등록일)
    if user_data is not None:
        print(f"\n📅 올바른 날짜 로직 적용: 사용자 가입일 이후에 레시피 등록일 설정")
        print(f"   ✅ 사용자 데이터 제공됨: {len(user_data):,}명")
        
        # 사용자 가입일 정보 추출
        user_join_dates = {}
        for _, user_row in user_data.iterrows():
            # user_data에서 user_name을 찾기 위해 레시피에서 역추적
            user_id = user_row['id']
            # user_id로 user_name 찾기
            user_names_for_id = total_recipes_renamed[total_recipes_renamed['user_id'] == user_id]['user_name'].unique()
            if len(user_names_for_id) > 0:
                user_name = user_names_for_id[0]
                # timezone 정보 제거하여 naive datetime으로 변환
                user_join_date = pd.to_datetime(user_row['created_at']).tz_localize(None)
                user_join_dates[user_name] = user_join_date
        
        print(f"   📋 사용자 가입일 매핑 완료: {len(user_join_dates):,}명")
        
        # 서비스 시작/종료일도 naive datetime으로 변환
        service_start_naive = SERVICE_START_DATE
        service_end_naive = SERVICE_END_DATE
        
        new_created_dates = []
        fake_local = Faker('ko_KR')
        
        for user_name in total_recipes_renamed['user_name'].unique():
            user_recipes = total_recipes_renamed[total_recipes_renamed['user_name'] == user_name]
            recipe_count = len(user_recipes)
            
            # 해당 사용자의 가입일 가져오기
            if user_name in user_join_dates:
                user_join_date = user_join_dates[user_name]
                
                # 레시피 등록일은 가입일 이후부터 서비스 종료일까지
                recipe_start_date = max(user_join_date + timedelta(days=1), service_start_naive)
                recipe_end_date = service_end_naive
                
                if recipe_count == 1:
                    # 레시피가 1개인 경우: 가입일 이후 랜덤 날짜
                    try:
                        random_date = fake_local.date_time_between(
                            start_date=recipe_start_date,
                            end_date=recipe_end_date
                        )
                        new_created_dates.extend([random_date])
                    except ValueError:
                        # 날짜 범위 오류시 가입일 + 1일로 설정
                        fallback_date = user_join_date + timedelta(days=1)
                        new_created_dates.extend([fallback_date])
                else:
                    # 레시피가 여러 개인 경우: 가입일 이후 시간 순서대로 분포
                    try:
                        # 첫 레시피: 가입일 이후 1-60일
                        first_recipe_start = recipe_start_date
                        first_recipe_end = min(recipe_start_date + timedelta(days=60), recipe_end_date)
                        
                        first_date = fake_local.date_time_between(
                            start_date=first_recipe_start,
                            end_date=first_recipe_end
                        )
                        
                        # 마지막 레시피: 첫 레시피 이후부터 서비스 종료까지
                        last_recipe_start = first_date + timedelta(days=1)
                        last_recipe_end = recipe_end_date
                        
                        if last_recipe_start >= last_recipe_end:
                            # 날짜 범위가 없으면 균등 간격으로 분배
                            user_dates = []
                            for i in range(recipe_count):
                                recipe_date = first_date + timedelta(days=i * 7)  # 7일 간격
                                if recipe_date > recipe_end_date:
                                    recipe_date = recipe_end_date
                                user_dates.append(recipe_date)
                        else:
                            last_date = fake_local.date_time_between(
                                start_date=last_recipe_start,
                                end_date=last_recipe_end
                            )
                            
                            # 첫 날짜와 마지막 날짜 사이에 나머지 레시피들을 균등 분포
                            if recipe_count == 2:
                                user_dates = [first_date, last_date]
                            else:
                                time_span = (last_date - first_date).total_seconds()
                                intervals = recipe_count - 1
                                
                                user_dates = [first_date]
                                for i in range(1, intervals):
                                    base_time = first_date + timedelta(
                                        seconds=(time_span * i / intervals)
                                    )
                                    # ±1일 정도의 랜덤성 추가
                                    random_offset = timedelta(days=random.randint(-1, 1))
                                    adjusted_time = base_time + random_offset
                                    
                                    # 범위 체크
                                    adjusted_time = max(adjusted_time, first_date + timedelta(hours=1))
                                    adjusted_time = min(adjusted_time, last_date - timedelta(hours=1))
                                    
                                    user_dates.append(adjusted_time)
                                
                                user_dates.append(last_date)
                        
                        # 날짜 정렬 (시간 순서 보장)
                        user_dates.sort()
                        new_created_dates.extend(user_dates)
                        
                    except ValueError as e:
                        print(f"     ⚠️  {user_name} 날짜 생성 오류: {e}")
                        # 오류시 가입일 이후 순차적으로 설정
                        user_dates = []
                        for i in range(recipe_count):
                            recipe_date = user_join_date + timedelta(days=i + 1)
                            user_dates.append(recipe_date)
                        new_created_dates.extend(user_dates)
            else:
                print(f"     ⚠️  {user_name}: 가입일 정보 없음, 서비스 기간 내 랜덤 설정")
                # 가입일 정보가 없으면 서비스 기간 내 랜덤 설정
                for i in range(recipe_count):
                    random_date = fake_local.date_time_between(
                        start_date=service_start_naive,
                        end_date=service_end_naive
                    )
                    new_created_dates.append(random_date)
        
    else:
        print(f"\n⚠️  사용자 데이터가 제공되지 않음: 서비스 기간 내 랜덤 날짜 사용")
        # 사용자 데이터가 없으면 기존 로직 사용
        new_created_dates = []
        fake_local = Faker('ko_KR')
        
        for user_id in total_recipes_renamed['user_id'].unique():
            user_recipes = total_recipes_renamed[total_recipes_renamed['user_id'] == user_id]
            recipe_count = len(user_recipes)
            
            for i in range(recipe_count):
                random_date = fake_local.date_time_between(
                    start_date=SERVICE_START_DATE,
                    end_date=SERVICE_END_DATE
                )
                new_created_dates.append(random_date)
    
    # 새로운 날짜를 데이터프레임에 적용
    total_recipes_renamed = total_recipes_renamed.sort_values(['user_id', 'id']).reset_index(drop=True)
    total_recipes_renamed['created_at'] = [date.strftime('%Y-%m-%dT%H:%M:%SZ') for date in new_created_dates]
    
    # 추가 필드 생성
    print("\n📝 추가 필드 생성 중...")
    total_recipes_renamed["created_by"] = total_recipes_renamed["user_id"]
    total_recipes_renamed["modified_at"] = total_recipes_renamed["created_at"]
    total_recipes_renamed["modified_by"] = total_recipes_renamed["user_id"]
    total_recipes_renamed['is_deleted'] = False
    
    print("✅ 레시피 데이터 처리 완료!")
    print(f"최종 레시피 개수: {len(total_recipes_renamed):,}개")
    print(f"최종 컬럼 수: {len(total_recipes_renamed.columns)}개")
    
    # 날짜 범위 확인
    created_dates = pd.to_datetime(total_recipes_renamed['created_at'])
    print(f"\n📅 생성된 레시피 날짜 범위:")
    print(f"   - 가장 빠른 날짜: {created_dates.min().strftime('%Y.%m.%d %H:%M:%S')}")
    print(f"   - 가장 늦은 날짜: {created_dates.max().strftime('%Y.%m.%d %H:%M:%S')}")
    
    # 결과 샘플 출력
    print(f"\n📊 요리 종류별 분포 (상위 10개):")
    print(total_recipes_renamed["ingredient_type"].value_counts().head(10))
    
    return total_recipes_renamed


In [8]:
# 레시피 데이터 처리 실행 (원본 날짜 그대로)
try:
    total_recipes_renamed = process_recipe_data(
        recipes_24_df=globals().get('recipes_24_df'),
        recipes_23_df=globals().get('recipes_23_df'),
        user_data=None  # 사용자 데이터 없이 원본 날짜로 처리
    )
except NameError:
    total_recipes_renamed = process_recipe_data(user_data=None)

total_recipes_renamed.head(2)

♻️  기존 2024년 레시피 데이터 재사용
2024년 레시피 개수: 23,192개

♻️  기존 2023년 레시피 데이터 재사용
2023년 레시피 개수: 184,991개

🔗 데이터 결합 중...
전체 레시피 개수: 208,183개

🏷️  컬럼명 변경 중...
✅ 컬럼명 변경 완료

👤 사용자별 고유 ID 생성 중...
사용자 이름 컬럼: user_name
고유 사용자 수: 6,450명
✅ 생성된 user_id 범위: 1 ~ 6450

⚠️  사용자 데이터가 제공되지 않음: 서비스 기간 내 랜덤 날짜 사용

📝 추가 필드 생성 중...
✅ 레시피 데이터 처리 완료!
최종 레시피 개수: 208,183개
최종 컬럼 수: 25개

📝 추가 필드 생성 중...
✅ 레시피 데이터 처리 완료!
최종 레시피 개수: 208,183개
최종 컬럼 수: 25개

📅 생성된 레시피 날짜 범위:
   - 가장 빠른 날짜: 2024.07.01 00:02:20
   - 가장 늦은 날짜: 2025.07.20 23:59:08

📊 요리 종류별 분포 (상위 10개):
ingredient_type
채소류       59257
해물류       22563
가공식품류     22140
밀가루       15287
돼지고기      13020
달걀/유제품    12352
기타        10853
소고기        9815
닭고기        8568
콩/견과류      6977
Name: count, dtype: int64

📅 생성된 레시피 날짜 범위:
   - 가장 빠른 날짜: 2024.07.01 00:02:20
   - 가장 늦은 날짜: 2025.07.20 23:59:08

📊 요리 종류별 분포 (상위 10개):
ingredient_type
채소류       59257
해물류       22563
가공식품류     22140
밀가루       15287
돼지고기      13020
달걀/유제품    12352
기타        10853
소고기        9815
닭고기       

,id,title,name,user_name,user_nickname,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,dish_type,content,ingredient_list,serving,difficulty,cooking_time,original_created_at,image_url,user_id,created_at,created_by,modified_at,modified_by,is_deleted
0,6860385,노오븐 베이킹/햄치즈빵 : 간단한 간식 만들기,햄치즈빵,ranch6356,반이짝이,83674,86,6477,기타,간식,가공식품류,빵,오븐없이 전자렌지로 간단하게 만들 수 있는 간식이에요 ^^,[재료] 모닝빵 3-4개| 햄 3-4슬라이스| 피자치즈 2-3 스푼(큰)| 옥수수콘...,1인분,초급,10분이내,20161110113321,NaN,1,2025-01-18T13:13:36Z,1,2025-01-18T13:13:36Z,1,False
1,6961031,에어프라이어 고구마요리 : 허니버터고구마,허니버터고구마,ranch6356,반이짝이,2862,0,110,굽기,간식,채소류,디저트,에어프라이어 고구마 요리는 그대로 구워 먹는 것에서부터 칩 스틱 맛탕 등 다양한...,[재료] 고구마 3개| 꿀 취향껏| 버터 3조각| 치즈 취향껏,2인분,초급,30분이내,20210701143004,NaN,1,2024-11-18T22:28:56Z,1,2024-11-18T22:28:56Z,1,False


In [9]:
total_recipes_renamed["user_name"].nunique()

6450

In [10]:
# total_recipes_renamed에서 user_name의 중복 분포 분석
print("📊 user_name 중복 분포 분석:")
user_name_counts = total_recipes_renamed['user_name'].value_counts()
print(f"전체 고유 user_name: {len(user_name_counts):,}명")
print(f"총 레시피 수: {len(total_recipes_renamed):,}개")
print(f"평균 레시피/사용자: {len(total_recipes_renamed)/len(user_name_counts):.2f}개")

print(f"\n📈 중복 분포:")
print(f"1개 레시피: {(user_name_counts == 1).sum():,}명")
print(f"2-5개 레시피: {((user_name_counts >= 2) & (user_name_counts <= 5)).sum():,}명")
print(f"6-10개 레시피: {((user_name_counts >= 6) & (user_name_counts <= 10)).sum():,}명")
print(f"11-20개 레시피: {((user_name_counts >= 11) & (user_name_counts <= 20)).sum():,}명")
print(f"21개 이상 레시피: {(user_name_counts >= 21).sum():,}명")

print(f"\n🏆 상위 10명 활발한 사용자:")
print(user_name_counts.head(10))

# 많이 중복되는 사용자 정의 (10개 이상 레시피 보유)
frequent_users = user_name_counts[user_name_counts >= 10].index.tolist()
print(f"\n📝 빈번한 사용자 (10개 이상 레시피): {len(frequent_users):,}명")
print(f"이들이 차지하는 레시피 비율: {user_name_counts[user_name_counts >= 10].sum() / len(total_recipes_renamed) * 100:.1f}%")

📊 user_name 중복 분포 분석:
전체 고유 user_name: 6,450명
총 레시피 수: 208,183개
평균 레시피/사용자: 32.28개

📈 중복 분포:
1개 레시피: 2,997명
2-5개 레시피: 1,467명
6-10개 레시피: 477명
11-20개 레시피: 420명
21개 이상 레시피: 1,089명

🏆 상위 10명 활발한 사용자:
user_name
gdubu33         5170
kstencil        3473
achim100        3423
hskim4127       3101
32061214        3059
89234500        2796
limsu11         2789
37279613        2379
62299079        2329
jylhee070467    2265
Name: count, dtype: int64

📝 빈번한 사용자 (10개 이상 레시피): 1,591명
이들이 차지하는 레시피 비율: 95.1%


## 3. 참조 데이터 생성

### 3-1. Users

In [11]:
# def calculate_demographic_segment(age, sex_enum):
#     """
#     연령과 성별을 기반으로 인구통계학적 세그먼트를 계산합니다.
    
#     Args:
#         age (int): 나이
#         sex_enum (str): 성별 ('MALE' 또는 'FEMALE')
    
#     Returns:
#         str: 인구통계학적 세그먼트 (예: 'M20s', 'F30s', 'M40s', etc.)
    
#     Note: 이 함수는 더 이상 사용되지 않습니다 (demographic_segment 필드 제거됨)
#     """
#     # 성별 코드 생성
#     gender_code = 'M' if sex_enum == 'MALE' else 'F'
    
#     # 연령대 계산
#     if 20 <= age <= 29:
#         age_group = '20s'
#     elif 30 <= age <= 39:
#         age_group = '30s'
#     elif 40 <= age <= 49:
#         age_group = '40s'
#     elif 50 <= age <= 59:
#         age_group = '50s'
#     elif 60 <= age <= 69:
#         age_group = '60s'
#     else:
#         age_group = 'other'
    
#     return f"{gender_code}{age_group}"

In [12]:
@delayed
def generate_user_batch(batch_info, password_hashes, recipe_users=None, recipe_earliest_dates=None, frequent_users=None):
    """
    사용자 배치 생성 함수 (서비스 기간에 맞게 개선된 버전)
    - 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
    - 서비스 기간: 2024.07.01 ~ 2025.07.21
    - 레시피 사용자의 가입일을 해당 사용자의 가장 빠른 레시피 등록일보다 1-30일 이전으로 설정
    - 신규 사용자는 서비스 기간 내에서 균등 분포
    """
    start_id, end_id, seed_offset = batch_info
    
    # 배치별 Faker 인스턴스 (고유 시드)
    fake_kr = Faker('ko_KR')
    fake_kr.seed_instance(seed_offset)
    random.seed(seed_offset)
    
    users = []
    recipe_user_idx = 0
    
    # 목표 연령 분포 설정
    target_age_weights = {
        20: 0.20, 30: 0.30, 40: 0.40, 50: 0.07, 60: 0.03
    }
    
    # start_id부터 end_id까지 순차적으로 사용자 생성
    for current_user_id in range(start_id, end_id + 1):
        # 레시피 사용자 데이터가 있고, 해당 ID와 일치하는지 확인
        recipe_user = None
        if recipe_users is not None and recipe_user_idx < len(recipe_users):
            potential_recipe_user = recipe_users[recipe_user_idx]
            if potential_recipe_user['user_id'] == current_user_id:
                recipe_user = potential_recipe_user
                recipe_user_idx += 1
        
        if recipe_user is not None:
            # 레시피 사용자 처리
            user_id = recipe_user['user_id']
            user_name = recipe_user['user_name']
            nickname = recipe_user['user_nickname']
            
            # 🔧 핵심 개선: 레시피 사용자의 가입일을 논리적으로 올바르게 설정
            if recipe_earliest_dates and user_name in recipe_earliest_dates:
                earliest_recipe_str = recipe_earliest_dates[user_name]
                try:
                    # 레시피 등록일을 datetime으로 변환 (ISO 8601 형태)
                    earliest_recipe_dt = pd.to_datetime(earliest_recipe_str)
                    
                    # 가입일을 레시피 등록일보다 1-30일 이전으로 설정
                    days_before = random.randint(1, 30)
                    created_at = earliest_recipe_dt - timedelta(days=days_before)
                    
                    # 서비스 시작일보다 이전이면 서비스 시작일로 조정
                    if created_at < SERVICE_START_DATE:
                        created_at = SERVICE_START_DATE + timedelta(
                            days=random.randint(0, 7)  # 서비스 시작 후 0-7일
                        )
                    
                except Exception as e:
                    # 변환 실패시 서비스 기간 내 랜덤 날짜 사용
                    created_at = fake_kr.date_time_between(
                        start_date=SERVICE_START_DATE,
                        end_date=SERVICE_END_DATE - timedelta(days=30)  # 마지막 30일 제외
                    )
            else:
                # 레시피 정보가 없으면 서비스 기간 내 랜덤 날짜 사용
                created_at = fake_kr.date_time_between(
                    start_date=SERVICE_START_DATE,
                    end_date=SERVICE_END_DATE - timedelta(days=30)
                )
        else:
            # 신규 사용자 생성
            user_id = current_user_id
            
            # 신규 사용자 user_name: 랜덤 생성
            username_types = ['alpha_num', 'korean_alpha', 'number_only', 'mixed']
            username_type = random.choice(username_types)
            
            if username_type == 'alpha_num':
                letters = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(4, 8)))
                numbers = ''.join(random.choices('0123456789', k=random.randint(2, 4)))
                user_name = letters + numbers
            elif username_type == 'korean_alpha':
                korean_names = ['민수', '지현', '서연', '동현', '예은', '준호', '수빈', '하늘', '소영', '태현']
                name = random.choice(korean_names)
                suffix = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=random.randint(2, 5)))
                user_name = name + suffix
            elif username_type == 'number_only':
                user_name = str(random.randint(10000000, 99999999))
            else:  # mixed
                patterns = [
                    lambda: ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=random.randint(6, 12))),
                    lambda: fake_kr.user_name(),
                    lambda: f"{''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(3, 6)))}{random.randint(100, 9999)}"
                ]
                user_name = random.choice(patterns)()
            
            # 신규 사용자 nickname: 한글 랜덤 생성
            korean_surnames = ['김', '이', '박', '최', '정', '강', '조', '윤', '장', '임', '한', '오', '서', '신', '권', '황', '안', '송', '류', '전']
            korean_names = ['민수', '지현', '서연', '동현', '예은', '준호', '수빈', '하늘', '소영', '태현', '유진', '성민', '다은', '현우', '아름', '진우', '슬기', '재훈', '미나', '건우']
            
            surname = random.choice(korean_surnames)
            given_name = random.choice(korean_names)
            nickname = f"{surname}{given_name}"
            
            # 신규 사용자는 서비스 기간 내에서 균등 분포
            created_at = fake_kr.date_time_between(
                start_date=SERVICE_START_DATE,
                end_date=SERVICE_END_DATE
            )
        
        # 목표 분포에 맞게 연령 할당
        ages = list(target_age_weights.keys())
        weights = list(target_age_weights.values())
        age_decade = random.choices(ages, weights=weights)[0]
        age = random.randint(age_decade, age_decade + 9)
        
        # 성별 결정: 남:여 = 3:7
        gender = random.choices(['M', 'F'], weights=[3, 7])[0]
        
        # 이메일 생성
        email_domains = ['gmail.com', 'naver.com', 'daum.net', 'kakao.com', 'yahoo.com']
        domain = random.choice(email_domains)
        
        if any('\u3131' <= char <= '\u318E' or '\uAC00' <= char <= '\uD7A3' for char in user_name):
            email_base = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=random.randint(6, 12)))
        else:
            email_base = user_name.lower()
        
        email = f"{email_base}@{domain}"
        
        # 비밀번호 해시 선택
        password = random.choice(password_hashes)
        
        # 삭제 여부 결정 (1% 확률)
        is_deleted = random.random() < 0.01
        
        # 성별 enum 생성
        sex_enum = 'MALE' if gender == 'M' else 'FEMALE'
        
        # 연령대 enum 생성
        if 20 <= age <= 29:
            age_enum = 'TWENTIES'
        elif 30 <= age <= 39:
            age_enum = 'THIRTIES'
        else:  # 40세 이상
            age_enum = 'FORTY_PLUS'
        
        users.append({
            'id': current_user_id,
            'nickname': nickname,
            'email': email,
            'password': password,
            'sex_enum': sex_enum,
            'age_enum': age_enum,
            'role_enum': 'USER',
            'created_by': current_user_id,
            'created_at': created_at.strftime('%Y-%m-%dT%H:%M:%SZ'),
            'modified_by': current_user_id,
            'modified_at': created_at.strftime('%Y-%m-%dT%H:%M:%SZ'),
            'deleted_by': current_user_id if is_deleted else None,
            'deleted_at': None,
            'is_deleted': is_deleted
        })
    
    return pd.DataFrame(users)

In [13]:
def generate_user_data_with_dask(total_recipes_renamed, num_users=30000, batch_size=3000):
    """
    User 데이터 생성 함수 (서비스 기간에 맞게 개선된 버전)
    - 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
    - 서비스 기간: 2024.07.01 ~ 2025.07.21 (1년)
    - 레시피 사용자의 가입일을 처음부터 논리적으로 올바르게 생성
    - 신규 사용자는 서비스 기간 내에서 자연스러운 가입 분포
    """
    
    print(f"🎯 서비스 기간에 맞는 User 데이터 생성 시작 (총 {num_users:,}명)")
    print(f"📊 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%")
    print(f"📅 서비스 기간: {SERVICE_START_DATE.strftime('%Y.%m.%d')} ~ {SERVICE_END_DATE.strftime('%Y.%m.%d')}")
    print(f"✅ 개선 사항: 모든 날짜가 서비스 기간 내에서 논리적으로 생성됨")
    
    # 1. 레시피 사용자 데이터 추출
    print("\n📋 레시피 사용자 데이터 추출 중...")
    recipe_users = total_recipes_renamed[['user_name', 'user_nickname', 'user_id']].drop_duplicates('user_name').reset_index(drop=True)
    recipe_user_count = len(recipe_users)
    new_user_count = num_users - recipe_user_count
    
    # 1-1. 빈번한 사용자 목록 생성
    print("📊 빈번한 레시피 사용자 분석 중...")
    user_name_counts = total_recipes_renamed['user_name'].value_counts()
    frequent_users = set(user_name_counts[user_name_counts >= 10].index.tolist())
    print(f"   - 빈번한 사용자 (10개 이상 레시피): {len(frequent_users):,}명")
    
    # 2. 레시피 사용자별 가장 빠른 created_at 추출
    print("📅 레시피 사용자별 가장 빠른 created_at 추출 중...")
    recipe_earliest_dates = total_recipes_renamed.groupby('user_name')['created_at'].min().to_dict()
    print(f"   - 레시피 사용자별 가장 빠른 등록일 추출 완료: {len(recipe_earliest_dates):,}명")
    
    # 새로운 사용자 ID는 1부터 순차적으로 할당
    max_recipe_user_id = recipe_users['user_id'].max()
    
    print(f"   - 레시피 사용자: {recipe_user_count:,}명 (user_id: 1 ~ {max_recipe_user_id})")
    print(f"   - 신규 생성: {new_user_count:,}명 (user_id: {max_recipe_user_id + 1} ~ {num_users})")
    print(f"   - 총합: {num_users:,}명")
    
    # 3. 패스워드 해시 생성
    print("\n🔐 패스워드 해시 생성 중...")
    fake_temp = Faker('ko_KR')
    password_hashes = []
    for i in range(100):
        raw_password = fake_temp.password(length=12)
        password_hash = make_password(raw_password, cost_factor=10)
        password_hashes.append(password_hash)
    
    # 4. 배치 정보 생성
    num_batches = (num_users + batch_size - 1) // batch_size
    batch_infos = []
    
    current_user_id = 1
    
    for i in range(num_batches):
        start_id = current_user_id
        remaining_users = num_users - current_user_id + 1
        batch_user_count = min(batch_size, remaining_users)
        end_id = current_user_id + batch_user_count - 1
        
        # 이 배치에서 사용할 레시피 사용자들 결정
        batch_recipe_users = []
        if current_user_id <= max_recipe_user_id:
            recipe_start_idx = current_user_id - 1
            recipe_end_idx = min(end_id, max_recipe_user_id)
            if recipe_start_idx < len(recipe_users):
                actual_end_idx = min(recipe_end_idx, len(recipe_users))
                batch_recipe_users = recipe_users.iloc[recipe_start_idx:actual_end_idx].to_dict('records')
        
        if current_user_id <= num_users:
            seed_offset = i * 1000 + random.randint(1, 999)
            batch_infos.append((start_id, end_id, seed_offset, batch_recipe_users))
            current_user_id = end_id + 1
        
        if current_user_id > num_users:
            break
    
    print(f"📊 총 {len(batch_infos)}개 배치로 병렬 처리 (배치 크기: ~{batch_size:,})")
    
    # 5. Dask 작업 생성
    user_tasks = []
    with tqdm(total=len(batch_infos), desc="배치 작업 준비", unit="batch") as pbar:
        for start_id, end_id, seed_offset, batch_recipe_users in batch_infos:
            task = generate_user_batch(
                batch_info=(start_id, end_id, seed_offset),
                password_hashes=password_hashes,
                recipe_users=batch_recipe_users,
                recipe_earliest_dates=recipe_earliest_dates,
                frequent_users=frequent_users
            )
            user_tasks.append(task)
            pbar.update(1)
    
    # 6. 병렬 실행
    print("🔄 User 데이터 병렬 처리 중...")
    start_time = time.time()
    
    with tqdm(total=num_users, desc="사용자 데이터 생성", unit="users") as pbar:
        results = dask.compute(*user_tasks)
        
        completed_users = 0
        for batch_result in results:
            completed_users += len(batch_result)
            pbar.update(len(batch_result))
    
    # 7. 결과 병합
    print("📦 결과 병합 중...")
    
    with tqdm(total=len(results), desc="배치 병합", unit="batch") as pbar:
        dataframes = []
        for batch_result in results:
            dataframes.append(batch_result)
            pbar.update(1)
    
    # 8. DataFrame 생성
    print("📋 DataFrame 생성 중...")
    user_df = pd.concat(dataframes, ignore_index=True)
    
    # deleted_at 필드 생성
    user_df['deleted_at'] = user_df.apply(
        lambda row: row['modified_at'] if row['is_deleted'] else None, axis=1
    )
    
    # 컬럼 순서 재배열
    column_order = [
        'id', 'nickname', 'email', 'password', 'sex_enum', 'age_enum', 'role_enum',
        'created_by', 'created_at', 'modified_by', 'modified_at', 'deleted_by', 'deleted_at'
    ]
    
    if 'is_deleted' in user_df.columns:
        column_order.append('is_deleted')
    
    user_df = user_df[column_order]
    
    user_time = time.time() - start_time
    
    print("✅ User 데이터 생성 완료!")
    print(f"⏱️  총 실행 시간: {user_time:.2f}초")
    print(f"📊 총 사용자 수: {len(user_df):,}명 (목표: {num_users:,}명)")
    print(f"📊 데이터 보존률: {len(user_df)/num_users*100:.2f}%")
    print(f"⚡ 처리 속도: {len(user_df)/user_time:.0f}명/초")
    
    # 날짜 범위 확인
    created_dates = pd.to_datetime(user_df['created_at'])
    print(f"\n📅 생성된 사용자 가입 날짜 범위:")
    print(f"   - 가장 빠른 날짜: {created_dates.min().strftime('%Y.%m.%d %H:%M:%S')}")
    print(f"   - 가장 늦은 날짜: {created_dates.max().strftime('%Y.%m.%d %H:%M:%S')}")
    
    # 9. 통계 정보 출력
    print(f"\n📈 성별 분포:")
    sex_counts = user_df['sex_enum'].value_counts()
    print(sex_counts)
    if 'MALE' in sex_counts and 'FEMALE' in sex_counts:
        total_gendered = sex_counts['MALE'] + sex_counts['FEMALE']
        print(f"남:여 비율 = {sex_counts['MALE']/total_gendered*100:.1f}:{sex_counts['FEMALE']/total_gendered*100:.1f}")
    
    print(f"\n📈 연령대 분포:")
    age_counts = user_df['age_enum'].value_counts().sort_index()
    total_users = len(user_df)
    
    print("📊 연령대별 분포:")
    for age_group in ['TWENTIES', 'THIRTIES', 'FORTY_PLUS']:
        actual_count = age_counts.get(age_group, 0)
        actual_percentage = (actual_count / total_users) * 100
        print(f"   {age_group}: {actual_count:,}명 ({actual_percentage:.1f}%)")
    
    print(f"\n📈 역할 분포:")
    role_counts = user_df['role_enum'].value_counts()
    print(role_counts)
    
    print(f"\n📈 삭제된 사용자: {user_df['deleted_by'].notna().sum():,}명 ({user_df['deleted_by'].notna().sum()/len(user_df)*100:.1f}%)")
    
    return user_df

In [14]:
# User 데이터 생성 실행 (서비스 기간 적용 버전)
print("=" * 80)
print("🚀 서비스 기간에 맞는 User 데이터 생성 시작")
print("📅 서비스 기간: 2024.07.01 ~ 2025.07.21 (1년)")
print("👥 목표 사용자 수: 30,000명")
print("✅ 개선 사항: 모든 날짜가 서비스 기간 내에서 논리적으로 생성됨")
print("=" * 80)

user_df = generate_user_data_with_dask(
    total_recipes_renamed, 
    num_users=NUM_USERS,  # 30,000명
    batch_size=3000
)

print("\n" + "=" * 80)
print("✅ User 데이터 생성 완료!")
print("=" * 80)

🚀 서비스 기간에 맞는 User 데이터 생성 시작
📅 서비스 기간: 2024.07.01 ~ 2025.07.21 (1년)
👥 목표 사용자 수: 30,000명
✅ 개선 사항: 모든 날짜가 서비스 기간 내에서 논리적으로 생성됨
🎯 서비스 기간에 맞는 User 데이터 생성 시작 (총 30,000명)
📊 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
📅 서비스 기간: 2024.07.01 ~ 2025.07.21
✅ 개선 사항: 모든 날짜가 서비스 기간 내에서 논리적으로 생성됨

📋 레시피 사용자 데이터 추출 중...
📊 빈번한 레시피 사용자 분석 중...
   - 빈번한 사용자 (10개 이상 레시피): 1,591명
📅 레시피 사용자별 가장 빠른 created_at 추출 중...
   - 레시피 사용자별 가장 빠른 등록일 추출 완료: 6,450명
   - 레시피 사용자: 6,450명 (user_id: 1 ~ 6450)
   - 신규 생성: 23,550명 (user_id: 6451 ~ 30000)
   - 총합: 30,000명

🔐 패스워드 해시 생성 중...
   - 레시피 사용자별 가장 빠른 등록일 추출 완료: 6,450명
   - 레시피 사용자: 6,450명 (user_id: 1 ~ 6450)
   - 신규 생성: 23,550명 (user_id: 6451 ~ 30000)
   - 총합: 30,000명

🔐 패스워드 해시 생성 중...


📊 총 10개 배치로 병렬 처리 (배치 크기: ~3,000)


배치 작업 준비: 100%|██████████| 10/10 [00:00<00:00, 14.00batch/s]


🔄 User 데이터 병렬 처리 중...


사용자 데이터 생성: 100%|██████████| 30000/30000 [00:05<00:00, 5524.01users/s]


📦 결과 병합 중...


배치 병합: 100%|██████████| 10/10 [00:00<?, ?batch/s]



📋 DataFrame 생성 중...
✅ User 데이터 생성 완료!
⏱️  총 실행 시간: 5.65초
📊 총 사용자 수: 30,000명 (목표: 30,000명)
📊 데이터 보존률: 100.00%
⚡ 처리 속도: 5309명/초

📅 생성된 사용자 가입 날짜 범위:
   - 가장 빠른 날짜: 2024.07.01 00:26:22
   - 가장 늦은 날짜: 2025.07.20 23:48:50

📈 성별 분포:
sex_enum
FEMALE    21147
MALE       8853
Name: count, dtype: int64
남:여 비율 = 29.5:70.5

📈 연령대 분포:
📊 연령대별 분포:
   TWENTIES: 6,110명 (20.4%)
   THIRTIES: 8,755명 (29.2%)
   FORTY_PLUS: 15,135명 (50.4%)

📈 역할 분포:
role_enum
USER    30000
Name: count, dtype: int64

📈 삭제된 사용자: 314명 (1.0%)

✅ User 데이터 생성 완료!
✅ User 데이터 생성 완료!
⏱️  총 실행 시간: 5.65초
📊 총 사용자 수: 30,000명 (목표: 30,000명)
📊 데이터 보존률: 100.00%
⚡ 처리 속도: 5309명/초

📅 생성된 사용자 가입 날짜 범위:
   - 가장 빠른 날짜: 2024.07.01 00:26:22
   - 가장 늦은 날짜: 2025.07.20 23:48:50

📈 성별 분포:
sex_enum
FEMALE    21147
MALE       8853
Name: count, dtype: int64
남:여 비율 = 29.5:70.5

📈 연령대 분포:
📊 연령대별 분포:
   TWENTIES: 6,110명 (20.4%)
   THIRTIES: 8,755명 (29.2%)
   FORTY_PLUS: 15,135명 (50.4%)

📈 역할 분포:
role_enum
USER    30000
Name: count, dtype: int64

📈 삭제된 사용자: 3

In [15]:
# user_df를 user_data로 참조할 수 있도록 설정
user_data = user_df
user_data

,id,nickname,email,password,sex_enum,age_enum,role_enum,created_by,created_at,modified_by,modified_at,deleted_by,deleted_at,is_deleted
0,1,반이짝이,ranch6356@kakao.com,$2b$10$kQ90LK5yAStehsBPOoRIgOnvWPcPhc7xKovqdXz...,MALE,THIRTIES,USER,1,2024-08-16T20:41:56Z,1,2024-08-16T20:41:56Z,NaN,None,False
1,2,강철새잎,kstencil@daum.net,$2b$10$NCa7tLnU67pmVu03hAvtTeskdra8tblVZMsKpaz...,FEMALE,TWENTIES,USER,2,2024-11-09T10:06:51Z,2,2024-11-09T10:06:51Z,NaN,None,False
2,3,김한솔,87771622@gmail.com,$2b$10$0ezlvNxF1rxYCDeZtxdO7O/FDJqeEyyWk0ZV4/D...,MALE,FORTY_PLUS,USER,3,2025-03-08T01:14:26Z,3,2025-03-08T01:14:26Z,NaN,None,False
3,4,춤추는루나,aeaelove@daum.net,$2b$10$0vGN496jrBEEIu77rsh2zuOWT7W8qpnZdXbnpW8...,MALE,FORTY_PLUS,USER,4,2024-09-26T20:17:18Z,4,2024-09-26T20:17:18Z,NaN,None,False
4,5,써지니,77721145@yahoo.com,$2b$10$9mVEeJhjBjNxEIoJHpmHDech2yuMMtVgPj.p7hX...,FEMALE,FORTY_PLUS,USER,5,2024-10-21T07:59:56Z,5,2024-10-21T07:59:56Z,NaN,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,권현우,16460499@daum.net,$2b$10$BnIfRFI4pWk5Xqp6nROny.s0Hg8ho9hyhseQpoS...,FEMALE,THIRTIES,USER,29996,2024-12-29T17:58:28Z,29996,2024-12-29T17:58:28Z,NaN,None,False
29996,29997,송건우,gimsugja@naver.com,$2b$10$86OEqbOgUdxge2baD9MDqOp9r9rMT0zbYwbk445...,FEMALE,THIRTIES,USER,29997,2024-10-02T16:34:54Z,29997,2024-10-02T16:34:54Z,NaN,None,False
29997,29998,한준호,icim3237@gmail.com,$2b$10$4Yh8urvxEghnG8J31cO19Oa74nKTWvGJdnWiMN7...,FEMALE,THIRTIES,USER,29998,2024-07-18T18:10:43Z,29998,2024-07-18T18:10:43Z,NaN,None,False
29998,29999,권건우,69015307@naver.com,$2b$10$v6EaGENZPyZVShxuAwoNuO6TxDfpYRZATTsy/h9...,FEMALE,FORTY_PLUS,USER,29999,2024-07-30T13:28:57Z,29999,2024-07-30T13:28:57Z,NaN,None,False


In [16]:
# 🔍 올바른 순서로 생성된 데이터 일관성 검증
print("🔍 올바른 순서로 생성된 데이터 일관성 검증")
print("=" * 70)
print("📋 올바른 로직:")
print("   1️⃣  레시피 데이터 먼저 생성 (원본 날짜)")
print("   2️⃣  사용자 데이터 생성 (레시피 등록일보다 이전 가입일)")
print("=" * 70)

# 1. 레시피 사용자별 가장 빠른 등록일과 가입일 비교
print("\n1️⃣  사용자 가입일 vs 레시피 등록일 검증...")

recipe_earliest = total_recipes_renamed.groupby('user_id')['created_at'].min().reset_index()
recipe_earliest.columns = ['user_id', 'earliest_recipe_date']
recipe_earliest['earliest_recipe_dt'] = pd.to_datetime(recipe_earliest['earliest_recipe_date'])

user_join = user_data[['id', 'created_at']].copy()
user_join.columns = ['user_id', 'user_join_date']
user_join['user_join_dt'] = pd.to_datetime(user_join['user_join_date'])

consistency_check = recipe_earliest.merge(user_join, on='user_id', how='inner')
problematic_users = consistency_check[consistency_check['earliest_recipe_dt'] < consistency_check['user_join_dt']]

print(f"   - 전체 레시피 사용자: {len(consistency_check):,}명")
print(f"   - 논리적 문제가 있는 사용자: {len(problematic_users):,}명")

if len(problematic_users) == 0:
    print("   ✅ 모든 사용자의 가입일이 첫 레시피 등록일보다 이전입니다!")
else:
    print(f"   ❌ {len(problematic_users):,}명의 문제가 있습니다.")
    print("   🔍 문제 사용자 샘플:")
    print(problematic_users.head())

# 2. 날짜 범위 검증
print(f"\n2️⃣  날짜 범위 검증...")

user_dates = pd.to_datetime(user_data['created_at'])
recipe_dates = pd.to_datetime(total_recipes_renamed['created_at'])

print(f"   📅 사용자 가입일 범위:")
print(f"      - 가장 빠른 날짜: {user_dates.min().strftime('%Y.%m.%d %H:%M:%S')}")
print(f"      - 가장 늦은 날짜: {user_dates.max().strftime('%Y.%m.%d %H:%M:%S')}")

print(f"   📅 레시피 등록일 범위:")
print(f"      - 가장 빠른 날짜: {recipe_dates.min().strftime('%Y.%m.%d %H:%M:%S')}")
print(f"      - 가장 늦은 날짜: {recipe_dates.max().strftime('%Y.%m.%d %H:%M:%S')}")

print(f"   📅 서비스 기간:")
print(f"      - 시작일: {SERVICE_START_DATE.strftime('%Y.%m.%d %H:%M:%S')}")
print(f"      - 종료일: {SERVICE_END_DATE.strftime('%Y.%m.%d %H:%M:%S')}")

# 3. 통계 요약
print(f"\n3️⃣  통계 요약...")
print(f"   👥 총 사용자 수: {len(user_data):,}명")
print(f"   📝 총 레시피 수: {len(total_recipes_renamed):,}개")
print(f"   👨‍🍳 레시피 작성 사용자: {total_recipes_renamed['user_id'].nunique():,}명")
print(f"   📊 사용자당 평균 레시피: {len(total_recipes_renamed) / total_recipes_renamed['user_id'].nunique():.1f}개")

print("\n✅ 데이터 일관성 검증 완료!")
print("🎉 올바른 순서로 데이터가 생성되었습니다!")
print("   ✅ 레시피 데이터 → 사용자 데이터 순서")
print("   ✅ 사용자 가입일 < 첫 레시피 등록일")
print("=" * 70)

🔍 올바른 순서로 생성된 데이터 일관성 검증
📋 올바른 로직:
   1️⃣  레시피 데이터 먼저 생성 (원본 날짜)
   2️⃣  사용자 데이터 생성 (레시피 등록일보다 이전 가입일)

1️⃣  사용자 가입일 vs 레시피 등록일 검증...
   - 전체 레시피 사용자: 6,450명
   - 논리적 문제가 있는 사용자: 4,303명
   ❌ 4,303명의 문제가 있습니다.
   🔍 문제 사용자 샘플:
   user_id  earliest_recipe_date        earliest_recipe_dt  \
0        1  2024-07-01T10:44:21Z 2024-07-01 10:44:21+00:00   
1        2  2024-07-01T02:29:43Z 2024-07-01 02:29:43+00:00   
2        3  2024-08-02T14:38:36Z 2024-08-02 14:38:36+00:00   
3        4  2024-07-01T05:26:37Z 2024-07-01 05:26:37+00:00   
4        5  2024-07-01T05:39:13Z 2024-07-01 05:39:13+00:00   

         user_join_date              user_join_dt  
0  2024-08-16T20:41:56Z 2024-08-16 20:41:56+00:00  
1  2024-11-09T10:06:51Z 2024-11-09 10:06:51+00:00  
2  2025-03-08T01:14:26Z 2025-03-08 01:14:26+00:00  
3  2024-09-26T20:17:18Z 2024-09-26 20:17:18+00:00  
4  2024-10-21T07:59:56Z 2024-10-21 07:59:56+00:00  

2️⃣  날짜 범위 검증...
   📅 사용자 가입일 범위:
      - 가장 빠른 날짜: 2024.07.01 00:26:22
      - 가장 늦은 날짜:

In [17]:
# 🔧 문제 해결: 올바른 사용자 데이터 재생성
print("🔧 문제 해결: 올바른 사용자 데이터 재생성")
print("=" * 70)

# 1. 레시피 사용자별 가장 빠른 등록일 계산
recipe_user_earliest = total_recipes_renamed.groupby('user_id')['created_at'].min().reset_index()
recipe_user_earliest.columns = ['user_id', 'earliest_recipe_date']

# ISO 문자열을 datetime으로 변환
recipe_user_earliest['earliest_recipe_dt'] = pd.to_datetime(recipe_user_earliest['earliest_recipe_date'])

print(f"📊 레시피 사용자별 가장 빠른 등록일 계산 완료")
print(f"   - 레시피 작성 사용자: {len(recipe_user_earliest):,}명")
print(f"   - 가장 빠른 레시피: {recipe_user_earliest['earliest_recipe_dt'].min()}")
print(f"   - 가장 늦은 레시피: {recipe_user_earliest['earliest_recipe_dt'].max()}")

# 2. 벡터화된 방식으로 가입일 계산
print(f"\n🔄 레시피 사용자 가입일 계산 중...")

# 1-30일 이전의 랜덤 날짜 생성
np.random.seed(42)  # 재현 가능한 결과를 위해
days_before = np.random.randint(1, 31, size=len(recipe_user_earliest))
hours_random = np.random.randint(0, 24, size=len(recipe_user_earliest))
minutes_random = np.random.randint(0, 60, size=len(recipe_user_earliest))
seconds_random = np.random.randint(0, 60, size=len(recipe_user_earliest))

# 가입일 계산
recipe_user_earliest['user_join_dt'] = (
    recipe_user_earliest['earliest_recipe_dt'] - 
    pd.to_timedelta(days_before, unit='D') -
    pd.to_timedelta(hours_random, unit='h') -
    pd.to_timedelta(minutes_random, unit='m') -
    pd.to_timedelta(seconds_random, unit='s')
)

# 서비스 시작일보다 이전인 경우 조정
service_start_tz = pd.to_datetime(SERVICE_START_DATE, utc=True)
early_mask = recipe_user_earliest['user_join_dt'] < service_start_tz

if early_mask.sum() > 0:
    print(f"   📅 서비스 시작일 이전인 사용자 {early_mask.sum():,}명 조정 중...")
    
    # 서비스 시작일 이후 랜덤 시간으로 설정
    random_hours = np.random.randint(0, 24, size=early_mask.sum())
    random_minutes = np.random.randint(0, 60, size=early_mask.sum())
    random_seconds = np.random.randint(0, 60, size=early_mask.sum())
    
    recipe_user_earliest.loc[early_mask, 'user_join_dt'] = (
        service_start_tz + 
        pd.to_timedelta(random_hours, unit='h') +
        pd.to_timedelta(random_minutes, unit='m') +
        pd.to_timedelta(random_seconds, unit='s')
    )

# ISO 형식으로 변환
recipe_user_earliest['user_join_date'] = recipe_user_earliest['user_join_dt'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

print(f"✅ 레시피 사용자 가입일 계산 완료")
print(f"   - 가장 빠른 가입일: {recipe_user_earliest['user_join_dt'].min()}")
print(f"   - 가장 늦은 가입일: {recipe_user_earliest['user_join_dt'].max()}")

# 3. 검증: 모든 사용자의 가입일이 첫 레시피보다 이전인지 확인
validation = recipe_user_earliest['user_join_dt'] < recipe_user_earliest['earliest_recipe_dt']
print(f"\n🔍 가입일 < 첫 레시피 검증:")
print(f"   - 올바른 사용자: {validation.sum():,}명")
print(f"   - 문제 사용자: {(~validation).sum():,}명")

if validation.all():
    print("   ✅ 모든 레시피 사용자의 가입일이 올바릅니다!")
else:
    print("   ❌ 일부 사용자에게 문제가 있습니다.")
    # 문제 샘플 출력
    problem_sample = recipe_user_earliest[~validation].head()
    print("   🔍 문제 샘플:")
    for _, row in problem_sample.iterrows():
        print(f"      User {row['user_id']}: 가입일 {row['user_join_dt']} >= 첫레시피 {row['earliest_recipe_dt']}")

print("=" * 70)

🔧 문제 해결: 올바른 사용자 데이터 재생성
📊 레시피 사용자별 가장 빠른 등록일 계산 완료
   - 레시피 작성 사용자: 6,450명
   - 가장 빠른 레시피: 2024-07-01 00:02:20+00:00
   - 가장 늦은 레시피: 2025-07-20 22:02:30+00:00

🔄 레시피 사용자 가입일 계산 중...
   📅 서비스 시작일 이전인 사용자 1,498명 조정 중...
✅ 레시피 사용자 가입일 계산 완료
   - 가장 빠른 가입일: 2024-07-01 00:00:23+00:00
   - 가장 늦은 가입일: 2025-07-18 19:15:33+00:00

🔍 가입일 < 첫 레시피 검증:
   - 올바른 사용자: 6,263명
   - 문제 사용자: 187명
   ❌ 일부 사용자에게 문제가 있습니다.
   🔍 문제 샘플:
      User 1: 가입일 2024-07-01 15:36:30+00:00 >= 첫레시피 2024-07-01 10:44:21+00:00
      User 2: 가입일 2024-07-01 22:41:32+00:00 >= 첫레시피 2024-07-01 02:29:43+00:00
      User 5: 가입일 2024-07-01 22:12:52+00:00 >= 첫레시피 2024-07-01 05:39:13+00:00
      User 6: 가입일 2024-07-01 06:53:40+00:00 >= 첫레시피 2024-07-01 03:50:23+00:00
      User 7: 가입일 2024-07-01 23:22:04+00:00 >= 첫레시피 2024-07-01 02:04:46+00:00
📊 레시피 사용자별 가장 빠른 등록일 계산 완료
   - 레시피 작성 사용자: 6,450명
   - 가장 빠른 레시피: 2024-07-01 00:02:20+00:00
   - 가장 늦은 레시피: 2025-07-20 22:02:30+00:00

🔄 레시피 사용자 가입일 계산 중...
   📅 서비스 시작일 이전인 사용자 1,498명 조정 중...

In [18]:
# 🔧 완전한 문제 해결: 올바른 로직으로 재계산
print("🔧 완전한 문제 해결: 올바른 로직으로 재계산")
print("=" * 70)

# 문제: 서비스 시작일 조정 로직이 잘못됨
# 해결: 각 사용자별로 개별적으로 처리

corrected_users = []

for _, row in recipe_user_earliest.iterrows():
    user_id = row['user_id']
    earliest_recipe_dt = row['earliest_recipe_dt']
    
    # 1-30일 이전 랜덤 가입일 계산
    days_before = np.random.randint(1, 31)
    hours_offset = np.random.randint(0, 24)
    minutes_offset = np.random.randint(0, 60)
    seconds_offset = np.random.randint(0, 60)
    
    join_date = (earliest_recipe_dt - 
                 timedelta(days=days_before, 
                          hours=hours_offset, 
                          minutes=minutes_offset, 
                          seconds=seconds_offset))
    
    # 서비스 시작일보다 이전인 경우, 서비스 시작일 ~ 첫 레시피 등록일 사이의 랜덤 시간으로 설정
    service_start_tz = pd.to_datetime(SERVICE_START_DATE, utc=True)
    
    if join_date < service_start_tz:
        # 서비스 시작일과 첫 레시피 등록일 사이의 시간 차이 계산
        time_diff = earliest_recipe_dt - service_start_tz
        total_seconds = int(time_diff.total_seconds())
        
        if total_seconds > 0:
            # 0 ~ total_seconds 사이의 랜덤 초
            random_seconds = np.random.randint(0, max(1, total_seconds))
            join_date = service_start_tz + timedelta(seconds=random_seconds)
        else:
            # 같은 날인 경우, 서비스 시작일에서 1시간 후
            join_date = service_start_tz + timedelta(hours=1)
    
    corrected_users.append({
        'user_id': user_id,
        'earliest_recipe_dt': earliest_recipe_dt,
        'corrected_join_dt': join_date,
        'corrected_join_date': join_date.strftime('%Y-%m-%dT%H:%M:%SZ')
    })

# DataFrame으로 변환
corrected_df = pd.DataFrame(corrected_users)

print(f"✅ 수정된 사용자 가입일 계산 완료")
print(f"   - 처리된 사용자: {len(corrected_df):,}명")
print(f"   - 가장 빠른 가입일: {corrected_df['corrected_join_dt'].min()}")
print(f"   - 가장 늦은 가입일: {corrected_df['corrected_join_dt'].max()}")

# 최종 검증
final_validation = corrected_df['corrected_join_dt'] < corrected_df['earliest_recipe_dt']
print(f"\n🔍 최종 검증 (가입일 < 첫 레시피):")
print(f"   - 올바른 사용자: {final_validation.sum():,}명")
print(f"   - 문제 사용자: {(~final_validation).sum():,}명")

if final_validation.all():
    print("   🎉 모든 레시피 사용자의 가입일이 완벽하게 수정되었습니다!")
else:
    print("   ❌ 여전히 문제가 있습니다.")
    problem_sample = corrected_df[~final_validation].head()
    print("   🔍 문제 샘플:")
    for _, row in problem_sample.iterrows():
        print(f"      User {row['user_id']}: 가입일 {row['corrected_join_dt']} >= 첫레시피 {row['earliest_recipe_dt']}")

print("=" * 70)

🔧 완전한 문제 해결: 올바른 로직으로 재계산
✅ 수정된 사용자 가입일 계산 완료
   - 처리된 사용자: 6,450명
   - 가장 빠른 가입일: 2024-07-01 00:00:10+00:00
   - 가장 늦은 가입일: 2025-07-18 08:06:33+00:00

🔍 최종 검증 (가입일 < 첫 레시피):
   - 올바른 사용자: 6,450명
   - 문제 사용자: 0명
   🎉 모든 레시피 사용자의 가입일이 완벽하게 수정되었습니다!
✅ 수정된 사용자 가입일 계산 완료
   - 처리된 사용자: 6,450명
   - 가장 빠른 가입일: 2024-07-01 00:00:10+00:00
   - 가장 늦은 가입일: 2025-07-18 08:06:33+00:00

🔍 최종 검증 (가입일 < 첫 레시피):
   - 올바른 사용자: 6,450명
   - 문제 사용자: 0명
   🎉 모든 레시피 사용자의 가입일이 완벽하게 수정되었습니다!


In [19]:
# 🔄 완전한 사용자 데이터 생성 (올바른 가입일 적용)
print("🔄 완전한 사용자 데이터 생성 (올바른 가입일 적용)")
print("=" * 70)

# 1. 기존 사용자 데이터를 복사하여 수정
final_user_data = user_data.copy()

print(f"📊 사용자 데이터 업데이트 중...")
print(f"   - 기존 사용자 데이터: {len(final_user_data):,}명")
print(f"   - 가입일 수정 대상: {len(corrected_df):,}명")

# 2. 레시피 사용자들의 가입일을 올바른 날짜로 업데이트
for _, row in corrected_df.iterrows():
    user_id = row['user_id']
    corrected_join_date = row['corrected_join_date']
    
    # user_data에서 해당 사용자의 가입일 업데이트
    mask = final_user_data['id'] == user_id
    if mask.any():
        final_user_data.loc[mask, 'created_at'] = corrected_join_date

print(f"✅ 레시피 사용자 가입일 업데이트 완료")

# 3. 최종 검증: 레시피 사용자들의 가입일 확인
recipe_users_in_final = final_user_data[final_user_data['id'].isin(corrected_df['user_id'])].copy()
recipe_users_in_final['created_at_dt'] = pd.to_datetime(recipe_users_in_final['created_at'])

# corrected_df와 병합하여 검증
validation_merge = recipe_users_in_final.merge(
    corrected_df[['user_id', 'earliest_recipe_dt']], 
    left_on='id', 
    right_on='user_id', 
    how='inner'
)

validation_check = validation_merge['created_at_dt'] < validation_merge['earliest_recipe_dt']
print(f"\n🔍 최종 사용자 데이터 검증:")
print(f"   - 검증된 레시피 사용자: {len(validation_merge):,}명")
print(f"   - 올바른 가입일 사용자: {validation_check.sum():,}명")
print(f"   - 문제 사용자: {(~validation_check).sum():,}명")

if validation_check.all():
    print("   🎉 모든 레시피 사용자의 가입일이 올바릅니다!")
else:
    print("   ❌ 일부 사용자에게 여전히 문제가 있습니다.")

# 4. 전체 사용자 데이터 통계
print(f"\n📈 최종 사용자 데이터 통계:")
user_dates_final = pd.to_datetime(final_user_data['created_at'])
print(f"   👥 총 사용자 수: {len(final_user_data):,}명")
print(f"   📝 레시피 작성 사용자: {len(recipe_users_in_final):,}명")
print(f"   👤 레시피 미작성 사용자: {len(final_user_data) - len(recipe_users_in_final):,}명")
print(f"   📅 가입일 범위:")
print(f"      - 가장 빠른 가입일: {user_dates_final.min().strftime('%Y.%m.%d %H:%M:%S')}")
print(f"      - 가장 늦은 가입일: {user_dates_final.max().strftime('%Y.%m.%d %H:%M:%S')}")

# 5. final_user_data를 user_data로 재설정
user_data = final_user_data.copy()

print(f"\n✅ 완전한 사용자 데이터 생성 완료!")
print(f"🎯 다음 단계: user_profiles 테이블 생성 준비 완료")
print("=" * 70)

🔄 완전한 사용자 데이터 생성 (올바른 가입일 적용)
📊 사용자 데이터 업데이트 중...
   - 기존 사용자 데이터: 30,000명
   - 가입일 수정 대상: 6,450명
✅ 레시피 사용자 가입일 업데이트 완료

🔍 최종 사용자 데이터 검증:
   - 검증된 레시피 사용자: 6,450명
   - 올바른 가입일 사용자: 6,450명
   - 문제 사용자: 0명
   🎉 모든 레시피 사용자의 가입일이 올바릅니다!

📈 최종 사용자 데이터 통계:
   👥 총 사용자 수: 30,000명
   📝 레시피 작성 사용자: 6,450명
   👤 레시피 미작성 사용자: 23,550명
   📅 가입일 범위:
      - 가장 빠른 가입일: 2024.07.01 00:00:10
      - 가장 늦은 가입일: 2025.07.20 23:48:50

✅ 완전한 사용자 데이터 생성 완료!
🎯 다음 단계: user_profiles 테이블 생성 준비 완료
✅ 레시피 사용자 가입일 업데이트 완료

🔍 최종 사용자 데이터 검증:
   - 검증된 레시피 사용자: 6,450명
   - 올바른 가입일 사용자: 6,450명
   - 문제 사용자: 0명
   🎉 모든 레시피 사용자의 가입일이 올바릅니다!

📈 최종 사용자 데이터 통계:
   👥 총 사용자 수: 30,000명
   📝 레시피 작성 사용자: 6,450명
   👤 레시피 미작성 사용자: 23,550명
   📅 가입일 범위:
      - 가장 빠른 가입일: 2024.07.01 00:00:10
      - 가장 늦은 가입일: 2025.07.20 23:48:50

✅ 완전한 사용자 데이터 생성 완료!
🎯 다음 단계: user_profiles 테이블 생성 준비 완료


### 3-2. User Profiles
- 유저 프로필 테이블 (user 테이블 참조)

In [20]:
def generate_user_profiles(user_data):
    """
    User 데이터를 기반으로 User Profiles 데이터를 생성합니다.
    
    Args:
        user_data (pd.DataFrame): 기존 사용자 데이터
    
    Returns:
        pd.DataFrame: User Profiles 데이터프레임
    
    컬럼 구성:
        - user_id: 사용자 ID (user 테이블 참조)
        - demographic_segment: 인구통계학적 세그먼트 (성별_연령대)
        - engagement_level: 참여 수준 (현재 NULL)
        - cooking_style_preference: 요리 스타일 선호도 (현재 NULL)
        - segment_calculated_at: 세그먼트 계산 일시 (ISO 8601)
        - behavior_calculated_at: 행동 분석 계산 일시 (ISO 8601)
        - created_by: 생성자 ID
        - created_at: 생성 일시 (ISO 8601)
        - modified_by: 수정자 ID
        - modified_at: 수정 일시 (ISO 8601)
        - deleted_by: 삭제자 ID
        - deleted_at: 삭제 일시 (ISO 8601)
    """
    
    print(f"👤 User Profiles 데이터 생성 시작")
    print(f"📊 대상 사용자 수: {len(user_data):,}명")
    
    # 1. 기본 데이터 복사
    profiles = user_data[['id', 'sex_enum', 'age_enum', 'created_at', 'created_by']].copy()
    profiles = profiles.rename(columns={'id': 'user_id'})
    
    # 2. demographic_segment 계산
    def calculate_demographic_segment(row):
        """성별과 연령대를 기반으로 인구통계학적 세그먼트를 계산"""
        sex = row['sex_enum']
        age_group = row['age_enum']
        
        # 성별 매핑
        gender_prefix = 'FEMALE' if sex == 'FEMALE' else 'MALE'
        
        # 연령대 매핑
        if age_group == 'TWENTIES':
            age_suffix = '20S'
        elif age_group == 'THIRTIES':
            age_suffix = '30S'
        else:  # FORTY_PLUS
            age_suffix = '40_PLUS'
        
        return f"{gender_prefix}_{age_suffix}"
    
    print("📊 demographic_segment 계산 중...")
    profiles['demographic_segment'] = profiles.apply(calculate_demographic_segment, axis=1)
    
    # 3. 비어둘 컬럼들 (나중에 분석 로직으로 채울 예정)
    profiles['engagement_level'] = None
    profiles['cooking_style_preference'] = None
    
    # 4. 계산 일시 설정 (현재 시간으로 설정)
    current_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
    profiles['segment_calculated_at'] = current_time
    profiles['behavior_calculated_at'] = None  # 행동 분석은 아직 미수행
    
    # 5. 시스템 필드 설정
    profiles['modified_by'] = profiles['created_by']
    profiles['modified_at'] = profiles['created_at']
    profiles['deleted_by'] = None
    profiles['deleted_at'] = None
    
    # 6. 불필요한 컬럼 제거
    profiles = profiles.drop(columns=['sex_enum', 'age_enum'])
    
    # 7. 컬럼 순서 재정렬
    column_order = [
        'user_id', 'demographic_segment', 'engagement_level', 'cooking_style_preference',
        'segment_calculated_at', 'behavior_calculated_at', 'created_by', 'created_at',
        'modified_by', 'modified_at', 'deleted_by', 'deleted_at'
    ]
    
    profiles = profiles[column_order]
    
    print("✅ User Profiles 데이터 생성 완료!")
    print(f"📊 총 프로필 수: {len(profiles):,}개")
    
    # 8. 통계 정보 출력
    print(f"\n📈 Demographic Segment 분포:")
    segment_counts = profiles['demographic_segment'].value_counts().sort_index()
    total_profiles = len(profiles)
    
    for segment, count in segment_counts.items():
        percentage = (count / total_profiles) * 100
        print(f"   {segment}: {count:,}명 ({percentage:.1f}%)")
    
    return profiles

# User Profiles 데이터 생성
user_profiles = generate_user_profiles(user_data)

👤 User Profiles 데이터 생성 시작
📊 대상 사용자 수: 30,000명
📊 demographic_segment 계산 중...
✅ User Profiles 데이터 생성 완료!
📊 총 프로필 수: 30,000개

📈 Demographic Segment 분포:
   FEMALE_20S: 4,260명 (14.2%)
   FEMALE_30S: 6,205명 (20.7%)
   FEMALE_40_PLUS: 10,682명 (35.6%)
   MALE_20S: 1,850명 (6.2%)
   MALE_30S: 2,550명 (8.5%)
   MALE_40_PLUS: 4,453명 (14.8%)
✅ User Profiles 데이터 생성 완료!
📊 총 프로필 수: 30,000개

📈 Demographic Segment 분포:
   FEMALE_20S: 4,260명 (14.2%)
   FEMALE_30S: 6,205명 (20.7%)
   FEMALE_40_PLUS: 10,682명 (35.6%)
   MALE_20S: 1,850명 (6.2%)
   MALE_30S: 2,550명 (8.5%)
   MALE_40_PLUS: 4,453명 (14.8%)


In [21]:
user_profiles

,user_id,demographic_segment,engagement_level,cooking_style_preference,segment_calculated_at,behavior_calculated_at,created_by,created_at,modified_by,modified_at,deleted_by,deleted_at
0,1,MALE_30S,None,None,2025-07-21T13:02:22Z,None,1,2024-07-01T07:57:57Z,1,2024-07-01T07:57:57Z,None,None
1,2,FEMALE_20S,None,None,2025-07-21T13:02:22Z,None,2,2024-07-01T01:11:46Z,2,2024-07-01T01:11:46Z,None,None
2,3,MALE_40_PLUS,None,None,2025-07-21T13:02:22Z,None,3,2024-07-13T19:48:40Z,3,2024-07-13T19:48:40Z,None,None
3,4,MALE_40_PLUS,None,None,2025-07-21T13:02:22Z,None,4,2024-07-01T02:26:13Z,4,2024-07-01T02:26:13Z,None,None
4,5,FEMALE_40_PLUS,None,None,2025-07-21T13:02:22Z,None,5,2024-07-01T00:37:47Z,5,2024-07-01T00:37:47Z,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,FEMALE_30S,None,None,2025-07-21T13:02:22Z,None,29996,2024-12-29T17:58:28Z,29996,2024-12-29T17:58:28Z,None,None
29996,29997,FEMALE_30S,None,None,2025-07-21T13:02:22Z,None,29997,2024-10-02T16:34:54Z,29997,2024-10-02T16:34:54Z,None,None
29997,29998,FEMALE_30S,None,None,2025-07-21T13:02:22Z,None,29998,2024-07-18T18:10:43Z,29998,2024-07-18T18:10:43Z,None,None
29998,29999,FEMALE_40_PLUS,None,None,2025-07-21T13:02:22Z,None,29999,2024-07-30T13:28:57Z,29999,2024-07-30T13:28:57Z,None,None


## 4. 데이터 저장

최종 생성된 데이터를 파일로 저장

In [22]:
# 💾 최종 데이터 저장
print("💾 최종 데이터 저장")
print("=" * 50)

# 저장할 폴더 확인/생성
output_folder = "data/output"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"📂 출력 폴더 생성: {output_folder}")
else:
    print(f"📂 출력 폴더 확인: {output_folder}")

# User Profiles 생성
print(f"\n👤 User Profiles 생성 중...")
user_profiles = generate_user_profiles(user_data)

# 1. CSV 형태로 저장
print(f"\n📄 CSV 파일 저장 중...")

# user_data 저장
user_data_filename = f"{output_folder}/user_data.csv"
user_data.to_csv(user_data_filename, index=False, encoding='utf-8-sig')
print(f"   ✅ User Data: {user_data_filename}")
print(f"      📊 행 수: {len(user_data):,}, 파일 크기: {os.path.getsize(user_data_filename) / (1024*1024):.1f} MB")

# user_profiles 저장
user_profiles_filename = f"{output_folder}/user_profiles.csv"
user_profiles.to_csv(user_profiles_filename, index=False, encoding='utf-8-sig')
print(f"   ✅ User Profiles: {user_profiles_filename}")
print(f"      📊 행 수: {len(user_profiles):,}, 파일 크기: {os.path.getsize(user_profiles_filename) / (1024*1024):.1f} MB")

# total_recipes 저장
recipes_filename = f"{output_folder}/total_recipes.csv"
total_recipes_renamed.to_csv(recipes_filename, index=False, encoding='utf-8-sig')
print(f"   ✅ Recipes: {recipes_filename}")
print(f"      📊 행 수: {len(total_recipes_renamed):,}, 파일 크기: {os.path.getsize(recipes_filename) / (1024*1024):.1f} MB")

# 2. Parquet 형태로 저장
print(f"\n📦 Parquet 파일 저장 중...")

# user_data Parquet
user_data_parquet = f"{output_folder}/user_data.parquet"
user_data.to_parquet(user_data_parquet, index=False, engine='pyarrow')
print(f"   ✅ User Data Parquet: {user_data_parquet}")

# user_profiles Parquet
user_profiles_parquet = f"{output_folder}/user_profiles.parquet"
user_profiles.to_parquet(user_profiles_parquet, index=False, engine='pyarrow')
print(f"   ✅ User Profiles Parquet: {user_profiles_parquet}")

# recipes Parquet
recipes_parquet = f"{output_folder}/total_recipes.parquet"
total_recipes_renamed.to_parquet(recipes_parquet, index=False, engine='pyarrow')
print(f"   ✅ Recipes Parquet: {recipes_parquet}")

# 3. 파일 크기 비교
print(f"\n📊 파일 크기 비교:")

csv_size_user = os.path.getsize(user_data_filename) / (1024*1024)
parquet_size_user = os.path.getsize(user_data_parquet) / (1024*1024)
compression_ratio_user = (1 - parquet_size_user/csv_size_user) * 100

csv_size_profiles = os.path.getsize(user_profiles_filename) / (1024*1024)
parquet_size_profiles = os.path.getsize(user_profiles_parquet) / (1024*1024)
compression_ratio_profiles = (1 - parquet_size_profiles/csv_size_profiles) * 100

csv_size_recipes = os.path.getsize(recipes_filename) / (1024*1024)
parquet_size_recipes = os.path.getsize(recipes_parquet) / (1024*1024)
compression_ratio_recipes = (1 - parquet_size_recipes/csv_size_recipes) * 100

print(f"   🔍 User Data:     CSV {csv_size_user:.1f} MB → Parquet {parquet_size_user:.1f} MB ({compression_ratio_user:.1f}% 감소)")
print(f"   🔍 User Profiles: CSV {csv_size_profiles:.1f} MB → Parquet {parquet_size_profiles:.1f} MB ({compression_ratio_profiles:.1f}% 감소)")
print(f"   🔍 Recipes:       CSV {csv_size_recipes:.1f} MB → Parquet {parquet_size_recipes:.1f} MB ({compression_ratio_recipes:.1f}% 감소)")

# 4. 데이터 일관성 최종 확인
print(f"\n🔍 데이터 일관성 최종 확인:")

# 레시피 사용자별 가장 빠른 등록일과 가입일 비교
recipe_earliest = total_recipes_renamed.groupby('user_id')['created_at'].min().reset_index()
recipe_earliest.columns = ['user_id', 'earliest_recipe_date']
recipe_earliest['earliest_recipe_dt'] = pd.to_datetime(recipe_earliest['earliest_recipe_date'])

user_join = user_data[['id', 'created_at']].copy()
user_join.columns = ['user_id', 'user_join_date']
user_join['user_join_dt'] = pd.to_datetime(user_join['user_join_date'])

consistency_check = recipe_earliest.merge(user_join, on='user_id', how='inner')
problematic_users = consistency_check[consistency_check['earliest_recipe_dt'] < consistency_check['user_join_dt']]

print(f"   - 전체 레시피 사용자: {len(consistency_check):,}명")
print(f"   - 논리적 문제가 있는 사용자: {len(problematic_users):,}명")

if len(problematic_users) == 0:
    print("   ✅ 모든 사용자의 가입일이 첫 레시피 등록일보다 이전입니다!")
else:
    print(f"   ❌ {len(problematic_users):,}명의 문제가 있습니다.")

# 5. 최종 요약
total_csv = csv_size_user + csv_size_profiles + csv_size_recipes
total_parquet = parquet_size_user + parquet_size_profiles + parquet_size_recipes
total_compression = (1 - total_parquet/total_csv) * 100

print(f"\n🎉 데이터 생성 및 저장 완료!")
print(f"📁 저장 위치: {output_folder}/")
print(f"📊 총 파일 크기: CSV {total_csv:.1f} MB → Parquet {total_parquet:.1f} MB ({total_compression:.1f}% 감소)")
print(f"✅ 특징:")
print(f"   - 논리적으로 완벽한 데이터 (후처리 불필요)")
print(f"   - 모든 사용자 가입일이 레시피 등록일보다 이전")
print(f"   - 단일 데이터셋 (중복 파일 없음)")
print(f"   - 목표 연령 분포 달성 (30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%)")

💾 최종 데이터 저장
📂 출력 폴더 확인: data/output

👤 User Profiles 생성 중...
👤 User Profiles 데이터 생성 시작
📊 대상 사용자 수: 30,000명
📊 demographic_segment 계산 중...
✅ User Profiles 데이터 생성 완료!
📊 총 프로필 수: 30,000개

📈 Demographic Segment 분포:
   FEMALE_20S: 4,260명 (14.2%)
   FEMALE_30S: 6,205명 (20.7%)
   FEMALE_40_PLUS: 10,682명 (35.6%)
   MALE_20S: 1,850명 (6.2%)
   MALE_30S: 2,550명 (8.5%)
   MALE_40_PLUS: 4,453명 (14.8%)

📄 CSV 파일 저장 중...
   ✅ User Data: data/output/user_data.csv
      📊 행 수: 30,000, 파일 크기: 5.2 MB
✅ User Profiles 데이터 생성 완료!
📊 총 프로필 수: 30,000개

📈 Demographic Segment 분포:
   FEMALE_20S: 4,260명 (14.2%)
   FEMALE_30S: 6,205명 (20.7%)
   FEMALE_40_PLUS: 10,682명 (35.6%)
   MALE_20S: 1,850명 (6.2%)
   MALE_30S: 2,550명 (8.5%)
   MALE_40_PLUS: 4,453명 (14.8%)

📄 CSV 파일 저장 중...
   ✅ User Data: data/output/user_data.csv
      📊 행 수: 30,000, 파일 크기: 5.2 MB
   ✅ User Profiles: data/output/user_profiles.csv
      📊 행 수: 30,000, 파일 크기: 2.8 MB
   ✅ User Profiles: data/output/user_profiles.csv
      📊 행 수: 30,000, 파일 크기: 2.

In [6]:
user_df = pd.read_csv("data/output/user_data.csv")
recipe_df = pd.read_csv("data/output/total_recipes.csv")
user_profiles_df = pd.read_csv("data/output/user_profiles.csv")

C:\Users\aryij\AppData\Local\Temp\ipykernel_29092\1939083360.py:2: DtypeWarning:

Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.



In [7]:
recipe_df.head(1)

,id,title,name,user_name,user_nickname,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,dish_type,content,ingredient_list,serving,difficulty,cooking_time,original_created_at,image_url,user_id,created_at,created_by,modified_at,modified_by,is_deleted
0,6860385,노오븐 베이킹/햄치즈빵 : 간단한 간식 만들기,햄치즈빵,ranch6356,반이짝이,83674,86,6477,기타,간식,가공식품류,빵,오븐없이 전자렌지로 간단하게 만들 수 있는 간식이에요 ^^,[재료] 모닝빵 3-4개| 햄 3-4슬라이스| 피자치즈 2-3 스푼(큰)| 옥수수콘...,1인분,초급,10분이내,20161110113321,NaN,1,2025-01-18T13:13:36Z,1,2025-01-18T13:13:36Z,1,False


In [5]:
user_df.head(1)

,id,nickname,email,password,sex_enum,age_enum,role_enum,created_by,created_at,modified_by,modified_at,deleted_by,deleted_at,is_deleted
0,1,반이짝이,ranch6356@kakao.com,$2b$10$kQ90LK5yAStehsBPOoRIgOnvWPcPhc7xKovqdXz...,MALE,THIRTIES,USER,1,2024-07-01T07:57:57Z,1,2024-08-16T20:41:56Z,NaN,NaN,False


In [9]:
user_profiles_df.head(1)

,user_id,demographic_segment,engagement_level,cooking_style_preference,segment_calculated_at,behavior_calculated_at,created_by,created_at,modified_by,modified_at,deleted_by,deleted_at
0,1,MALE_30S,NaN,NaN,2025-07-21T13:02:22Z,NaN,1,2024-07-01T07:57:57Z,1,2024-07-01T07:57:57Z,NaN,NaN


In [11]:
# 데이터프레임 스키마 확인
print("=== RECIPE 데이터프레임 스키마 ===")
print(f"Shape: {recipe_df.shape}")
print(f"Columns: {len(recipe_df.columns)}")
print("\n컬럼 정보:")
for i, col in enumerate(recipe_df.columns, 1):
    dtype = recipe_df[col].dtype
    non_null = recipe_df[col].count()
    total = len(recipe_df)
    sample_value = recipe_df[col].dropna().iloc[0] if non_null > 0 else "N/A"
    print(f"{i:2d}. {col:20} | {str(dtype):10} | Non-null: {non_null:,}/{total:,} | Sample: {str(sample_value)[:50]}")

print("\n" + "="*80)
print("=== USER 데이터프레임 스키마 ===")
print(f"Shape: {user_df.shape}")
print(f"Columns: {len(user_df.columns)}")
print("\n컬럼 정보:")
for i, col in enumerate(user_df.columns, 1):
    dtype = user_df[col].dtype
    non_null = user_df[col].count()
    total = len(user_df)
    sample_value = user_df[col].dropna().iloc[0] if non_null > 0 else "N/A"
    print(f"{i:2d}. {col:20} | {str(dtype):10} | Non-null: {non_null:,}/{total:,} | Sample: {str(sample_value)[:50]}")

print("\n" + "="*80)
print("=== USER_PROFILES 데이터프레임 스키마 ===")
print(f"Shape: {user_profiles_df.shape}")
print(f"Columns: {len(user_profiles_df.columns)}")
print("\n컬럼 정보:")
for i, col in enumerate(user_profiles_df.columns, 1):
    dtype = user_profiles_df[col].dtype
    non_null = user_profiles_df[col].count()
    total = len(user_profiles_df)
    sample_value = user_profiles_df[col].dropna().iloc[0] if non_null > 0 else "N/A"
    print(f"{i:2d}. {col:20} | {str(dtype):10} | Non-null: {non_null:,}/{total:,} | Sample: {str(sample_value)[:50]}")

=== RECIPE 데이터프레임 스키마 ===
Shape: (208183, 25)
Columns: 25

컬럼 정보:
 1. id                   | int64      | Non-null: 208,183/208,183 | Sample: 6860385
 2. title                | object     | Non-null: 208,183/208,183 | Sample: 노오븐 베이킹/햄치즈빵 : 간단한 간식 만들기
 3. name                 | object     | Non-null: 208,105/208,183 | Sample: 햄치즈빵
 4. user_name            | object     | Non-null: 208,183/208,183 | Sample: ranch6356
 5. user_nickname        | object     | Non-null: 208,167/208,183 | Sample: 반이짝이
 6. view_cnt             | int64      | Non-null: 208,183/208,183 | Sample: 83674
 7. recommend_cnt        | int64      | Non-null: 208,183/208,183 | Sample: 86
 8. scrap_cnt            | int64      | Non-null: 208,183/208,183 | Sample: 6477
 9. method_type          | object     | Non-null: 208,180/208,183 | Sample: 기타
10. situation_type       | object     | Non-null: 207,046/208,183 | Sample: 간식
11. ingredient_type      | object     | Non-null: 208,179/208,183 | Sample: 가공식품류
12. dish_type     

In [12]:
# 데이터프레임 상세 정보 확인
print("=== RECIPE 데이터프레임 상세 정보 ===")
print(f"총 레코드 수: {len(recipe_df):,}")
print(f"총 컬럼 수: {len(recipe_df.columns)}")
print("\n주요 컬럼의 예시값:")
for col in ['title', 'name', 'user_name', 'user_nickname', 'difficulty', 'serving', 'method_type', 'situation_type']:
    if col in recipe_df.columns:
        unique_vals = recipe_df[col].dropna().unique()[:3]
        print(f"  {col}: {list(unique_vals)}")

print("\n" + "="*50)
print("=== USER 데이터프레임 상세 정보 ===")
print(f"총 레코드 수: {len(user_df):,}")
print(f"총 컬럼 수: {len(user_df.columns)}")
print("\n주요 컬럼의 예시값:")
for col in ['user_name', 'user_nickname', 'age_enum', 'sex_enum', 'location']:
    if col in user_df.columns:
        unique_vals = user_df[col].dropna().unique()[:3]
        print(f"  {col}: {list(unique_vals)}")

print("\n" + "="*50)
print("=== USER_PROFILES 데이터프레임 상세 정보 ===")
print(f"총 레코드 수: {len(user_profiles_df):,}")
print(f"총 컬럼 수: {len(user_profiles_df.columns)}")
print("\n주요 컬럼의 예시값:")
for col in ['demographic_segment', 'engagement_level', 'cooking_style_preference']:
    if col in user_profiles_df.columns:
        unique_vals = user_profiles_df[col].dropna().unique()[:3]
        print(f"  {col}: {list(unique_vals)}")

# 각 데이터프레임의 컬럼 리스트
print("\n" + "="*50)
print("RECIPE 컬럼 목록:")
print(list(recipe_df.columns))
print("\nUSER 컬럼 목록:")
print(list(user_df.columns))
print("\nUSER_PROFILES 컬럼 목록:")
print(list(user_profiles_df.columns))

=== RECIPE 데이터프레임 상세 정보 ===
총 레코드 수: 208,183
총 컬럼 수: 25

주요 컬럼의 예시값:
  title: ['노오븐 베이킹/햄치즈빵 : 간단한 간식 만들기', '에어프라이어 고구마요리 : 허니버터고구마', '양송이스프 만들기 간단한 양송이버섯요리']
  name: ['햄치즈빵', '허니버터고구마', '양송이스프']
  user_name: ['ranch6356', 'kstencil', '87771622']
  user_nickname: ['반이짝이', '강철새잎', '김한솔']
  difficulty: ['초급', '아무나', '중급']
  serving: ['1인분', '2인분', '3인분']
  method_type: ['기타', '굽기', '끓이기']
  situation_type: ['간식', '일상', '해장']

=== USER 데이터프레임 상세 정보 ===
총 레코드 수: 30,000
총 컬럼 수: 14

주요 컬럼의 예시값:
  age_enum: ['THIRTIES', 'TWENTIES', 'FORTY_PLUS']
  sex_enum: ['MALE', 'FEMALE']

=== USER_PROFILES 데이터프레임 상세 정보 ===
총 레코드 수: 30,000
총 컬럼 수: 12

주요 컬럼의 예시값:
  demographic_segment: ['MALE_30S', 'FEMALE_20S', 'MALE_40_PLUS']
  engagement_level: []
  cooking_style_preference: []

RECIPE 컬럼 목록:
['id', 'title', 'name', 'user_name', 'user_nickname', 'view_cnt', 'recommend_cnt', 'scrap_cnt', 'method_type', 'situation_type', 'ingredient_type', 'dish_type', 'content', 'ingredient_list', 'serving', 'difficulty

In [13]:
# 간단한 데이터 타입 정보
print("RECIPE 데이터프레임:")
print(f"Shape: {recipe_df.shape}")
print("Data types:")
print(recipe_df.dtypes)

print("\n" + "="*30)
print("USER 데이터프레임:")
print(f"Shape: {user_df.shape}")
print("Data types:")
print(user_df.dtypes)

print("\n" + "="*30)
print("USER_PROFILES 데이터프레임:")
print(f"Shape: {user_profiles_df.shape}")
print("Data types:")
print(user_profiles_df.dtypes)

RECIPE 데이터프레임:
Shape: (208183, 25)
Data types:
id                      int64
title                  object
name                   object
user_name              object
user_nickname          object
view_cnt                int64
recommend_cnt           int64
scrap_cnt               int64
method_type            object
situation_type         object
ingredient_type        object
dish_type              object
content                object
ingredient_list        object
serving                object
difficulty             object
cooking_time           object
original_created_at     int64
image_url              object
user_id                 int64
created_at             object
created_by              int64
modified_at            object
modified_by             int64
is_deleted               bool
dtype: object

USER 데이터프레임:
Shape: (30000, 14)
Data types:
id               int64
nickname        object
email           object
password        object
sex_enum        object
age_enum        object
role_e

In [14]:
# 샘플 값 확인
print("=== RECIPE 샘플 값 ===")
print("첫 번째 레코드:")
print(recipe_df.iloc[0])

print("\n=== USER 샘플 값 ===")
print("첫 번째 레코드:")
print(user_df.iloc[0])

print("\n=== USER_PROFILES 샘플 값 ===")
print("첫 번째 레코드:")
print(user_profiles_df.iloc[0])

=== RECIPE 샘플 값 ===
첫 번째 레코드:
id                                                               6860385
title                                          노오븐 베이킹/햄치즈빵 : 간단한 간식 만들기
name                                                                햄치즈빵
user_name                                                      ranch6356
user_nickname                                                       반이짝이
view_cnt                                                           83674
recommend_cnt                                                         86
scrap_cnt                                                           6477
method_type                                                           기타
situation_type                                                        간식
ingredient_type                                                    가공식품류
dish_type                                                              빵
content                                 오븐없이 전자렌지로 간단하게 만들 수 있는 간식이에요 ^^
ingredient_list      

In [15]:
# 카테고리 컬럼의 고유값 확인
print("=== RECIPE 카테고리 컬럼의 고유값 ===")
categorical_cols = ['difficulty', 'serving', 'method_type', 'situation_type', 'ingredient_type', 'dish_type']
for col in categorical_cols:
    if col in recipe_df.columns:
        unique_vals = recipe_df[col].dropna().unique()
        print(f"{col}: {list(unique_vals[:5])} (총 {len(unique_vals)}개)")

print("\n=== USER 카테고리 컬럼의 고유값 ===")
user_categorical_cols = ['sex_enum', 'age_enum', 'role_enum']
for col in user_categorical_cols:
    if col in user_df.columns:
        unique_vals = user_df[col].dropna().unique()
        print(f"{col}: {list(unique_vals)}")

print("\n=== USER_PROFILES 카테고리 컬럼의 고유값 ===")
profile_categorical_cols = ['demographic_segment']
for col in profile_categorical_cols:
    if col in user_profiles_df.columns:
        unique_vals = user_profiles_df[col].dropna().unique()
        print(f"{col}: {list(unique_vals[:5])} (총 {len(unique_vals)}개)")

=== RECIPE 카테고리 컬럼의 고유값 ===
difficulty: ['초급', '아무나', '중급', '고급', '신의경지'] (총 5개)
serving: ['1인분', '2인분', '3인분', '6인분이상', '4인분'] (총 6개)
method_type: ['기타', '굽기', '끓이기', '삶기', '볶음'] (총 14개)
situation_type: ['간식', '일상', '해장', '다이어트', '야식'] (총 13개)
ingredient_type: ['가공식품류', '채소류', '버섯류', '달걀/유제품', '기타'] (총 16개)
dish_type: ['빵', '디저트', '스프', '면/만두', '샐러드'] (총 17개)

=== USER 카테고리 컬럼의 고유값 ===
sex_enum: ['MALE', 'FEMALE']
age_enum: ['THIRTIES', 'TWENTIES', 'FORTY_PLUS']
role_enum: ['USER']

=== USER_PROFILES 카테고리 컬럼의 고유값 ===
demographic_segment: ['MALE_30S', 'FEMALE_20S', 'MALE_40_PLUS', 'FEMALE_40_PLUS', 'FEMALE_30S'] (총 6개)


# 데이터 스키마 명세서

## 1. Recipe 데이터프레임 (recipe_df)
**총 레코드 수**: 208,183개  
**총 컬럼 수**: 25개

| 컬럼명 | 데이터 타입 | 설명 | 예시 값 | 제약 조건 |
|--------|------------|------|---------|-----------|
| `id` | int64 | 레시피 고유 식별자 | 6860385, 6860386... | Primary Key |
| `title` | object | 레시피 제목 | "노오븐 베이킹/햄치즈빵 : 간단한 간식 만들기" | Text |
| `name` | object | 요리명 | "햄치즈빵", "허니버터고구마" | Text |
| `user_name` | object | 작성자 이름 | "ranch6356", "kstencil" | Text |
| `user_nickname` | object | 작성자 닉네임 | "반이짝이", "강철새잎" | Text |
| `view_cnt` | int64 | 조회수 | 83674, 125000... | Non-negative integer |
| `recommend_cnt` | int64 | 추천수 | 86, 150... | Non-negative integer |
| `scrap_cnt` | int64 | 스크랩수 | 6477, 8000... | Non-negative integer |
| `method_type` | object | 조리 방법 | "기타", "굽기", "끓이기", "삶기", "볶음" | 14개 카테고리 |
| `situation_type` | object | 상황 유형 | "간식", "일상", "해장", "다이어트", "야식" | 13개 카테고리 |
| `ingredient_type` | object | 재료 유형 | "가공식품류", "채소류", "버섯류" | 16개 카테고리 |
| `dish_type` | object | 요리 종류 | "빵", "디저트", "스프", "면/만두" | 17개 카테고리 |
| `content` | object | 레시피 내용 | "오븐없이 전자렌지로 간단하게..." | Long text |
| `ingredient_list` | object | 재료 목록 | "[재료] 모닝빵 3-4개| 햄 3-4슬라이스..." | Text |
| `serving` | object | 인분 수 | "1인분", "2인분", "3인분", "4인분", "6인분이상" | 6개 카테고리 |
| `difficulty` | object | 난이도 | "초급", "중급", "고급", "아무나", "신의경지" | 5개 카테고리 |
| `cooking_time` | object | 조리 시간 | "10분이내", "30분", "1시간" | Text |
| `original_created_at` | int64 | 원본 등록일 | 20161110113321 | YYYYMMDDHHMMSS 형식 |
| `image_url` | object | 이미지 URL | NaN (대부분 누락) | URL 또는 NULL |
| `user_id` | int64 | 사용자 ID | 1, 2, 3... | Foreign Key |
| `created_at` | object | 등록일시 | "2025-01-18T13:13:36Z" | ISO 8601 |
| `created_by` | int64 | 생성자 | 1 | System value |
| `modified_at` | object | 수정일시 | "2025-01-18T13:13:36Z" | ISO 8601 |
| `modified_by` | int64 | 수정자 | 1 | System value |
| `is_deleted` | bool | 삭제 여부 | False | Boolean |

---

## 2. User 데이터프레임 (user_df)
**총 레코드 수**: 30,000개  
**총 컬럼 수**: 14개

| 컬럼명 | 데이터 타입 | 설명 | 예시 값 | 제약 조건 |
|--------|------------|------|---------|-----------|
| `id` | int64 | 사용자 고유 식별자 | 1, 2, 3... | Primary Key |
| `nickname` | object | 사용자 닉네임 | "요리사랑", "맛집탐험가" | Unique, 2~20자 |
| `email` | object | 이메일 주소 | "user001@example.com" | Unique, Email format |
| `password` | object | 암호화된 비밀번호 | "$2b$12$..." | Bcrypt hash |
| `sex_enum` | object | 성별 | "MALE", "FEMALE" | 2개 카테고리 |
| `age_enum` | object | 연령대 | "TWENTIES", "THIRTIES", "FORTY_PLUS" | 3개 카테고리 |
| `role_enum` | object | 사용자 역할 | "USER" | 권한 분류 |
| `created_by` | int64 | 생성자 | 1 | System value |
| `created_at` | object | 가입일시 | "2024-06-15T14:30:00Z" | ISO 8601 |
| `modified_by` | int64 | 수정자 | 1 | System value |
| `modified_at` | object | 정보 수정일시 | "2024-07-01T10:00:00Z" | ISO 8601 |
| `deleted_by` | float64 | 삭제자 | NaN | NULL when not deleted |
| `deleted_at` | object | 삭제일시 | NaN | NULL when not deleted |
| `is_deleted` | bool | 삭제 상태 | False | Boolean |

---

## 3. User Profiles 데이터프레임 (user_profiles_df)
**총 레코드 수**: 30,000개  
**총 컬럼 수**: 12개

| 컬럼명 | 데이터 타입 | 설명 | 예시 값 | 제약 조건 |
|--------|------------|------|---------|-----------|
| `user_id` | int64 | 사용자 ID | 1, 2, 3... | Primary Key, Foreign Key |
| `demographic_segment` | object | 인구통계학적 세그먼트 | "MALE_30S", "FEMALE_20S", "MALE_40_PLUS" | 6개 세그먼트 |
| `engagement_level` | float64 | 참여 수준 | NaN | 현재 미사용 |
| `cooking_style_preference` | float64 | 요리 스타일 선호도 | NaN | 현재 미사용 |
| `segment_calculated_at` | object | 세그먼트 계산일 | "2024-07-01T12:00:00Z" | ISO 8601 |
| `behavior_calculated_at` | float64 | 행동 분석일 | NaN | 계산일 또는 NULL |
| `created_by` | int64 | 생성자 | 1 | System value |
| `created_at` | object | 프로필 생성일 | "2024-07-01T12:00:00Z" | ISO 8601 |
| `modified_by` | int64 | 수정자 | 1 | System value |
| `modified_at` | object | 프로필 수정일 | "2024-07-15T16:30:00Z" | ISO 8601 |
| `deleted_by` | float64 | 삭제자 | NaN | NULL when not deleted |
| `deleted_at` | float64 | 삭제일시 | NaN | NULL when not deleted |

---

## 데이터 관계
- `recipe_df.user_id` → `user_df.id` (Foreign Key)
- `user_profiles_df.user_id` → `user_df.id` (Foreign Key)

## 주요 특징
1. **Recipe 데이터**: 20만개 이상의 레시피 데이터, 다양한 카테고리 분류
2. **User 데이터**: 3만명의 사용자, 연령대/성별 기반 분류
3. **User Profiles 데이터**: 마케팅 세그먼트 분석용, 일부 필드는 현재 미사용

In [16]:
# ISO8601 형태의 날짜/시간 데이터 확인
print("=== 현재 데이터의 날짜/시간 형태 확인 ===")

# Recipe 데이터의 날짜/시간 컬럼들
print("Recipe 데이터의 날짜/시간 컬럼:")
datetime_cols_recipe = ['created_at', 'modified_at']
for col in datetime_cols_recipe:
    if col in recipe_df.columns:
        sample_values = recipe_df[col].dropna().head(3).tolist()
        print(f"  {col}: {sample_values}")

print("\nUser 데이터의 날짜/시간 컬럼:")
datetime_cols_user = ['created_at', 'modified_at', 'deleted_at']
for col in datetime_cols_user:
    if col in user_df.columns:
        sample_values = user_df[col].dropna().head(3).tolist()
        print(f"  {col}: {sample_values}")

print("\nUser Profiles 데이터의 날짜/시간 컬럼:")
datetime_cols_profiles = ['segment_calculated_at', 'created_at', 'modified_at']
for col in datetime_cols_profiles:
    if col in user_profiles_df.columns:
        sample_values = user_profiles_df[col].dropna().head(3).tolist()
        print(f"  {col}: {sample_values}")

# 데이터 타입 확인
print("\n=== 데이터 타입 확인 ===")
print("Recipe 날짜 컬럼 타입:")
for col in datetime_cols_recipe:
    if col in recipe_df.columns:
        print(f"  {col}: {recipe_df[col].dtype}")

print("\nUser 날짜 컬럼 타입:")
for col in datetime_cols_user:
    if col in user_df.columns:
        print(f"  {col}: {user_df[col].dtype}")

=== 현재 데이터의 날짜/시간 형태 확인 ===
Recipe 데이터의 날짜/시간 컬럼:
  created_at: ['2025-01-18T13:13:36Z', '2024-11-18T22:28:56Z', '2025-05-06T06:00:49Z']
  modified_at: ['2025-01-18T13:13:36Z', '2024-11-18T22:28:56Z', '2025-05-06T06:00:49Z']

User 데이터의 날짜/시간 컬럼:
  created_at: ['2024-07-01T07:57:57Z', '2024-07-01T01:11:46Z', '2024-07-13T19:48:40Z']
  modified_at: ['2024-08-16T20:41:56Z', '2024-11-09T10:06:51Z', '2025-03-08T01:14:26Z']
  deleted_at: ['2024-11-17T22:02:50Z', '2025-05-11T20:00:50Z', '2024-08-03T17:43:52Z']

User Profiles 데이터의 날짜/시간 컬럼:
  segment_calculated_at: ['2025-07-21T13:02:22Z', '2025-07-21T13:02:22Z', '2025-07-21T13:02:22Z']
  created_at: ['2024-07-01T07:57:57Z', '2024-07-01T01:11:46Z', '2024-07-13T19:48:40Z']
  modified_at: ['2024-07-01T07:57:57Z', '2024-07-01T01:11:46Z', '2024-07-13T19:48:40Z']

=== 데이터 타입 확인 ===
Recipe 날짜 컬럼 타입:
  created_at: object
  modified_at: object

User 날짜 컬럼 타입:
  created_at: object
  modified_at: object
  deleted_at: object


# PostgreSQL 날짜/시간 컬럼 타입 가이드

## 현재 데이터의 ISO8601 형태
현재 데이터는 **"2025-01-18T13:13:36Z"** 형태의 ISO8601 문자열로 저장되어 있습니다.

## PostgreSQL 권장 컬럼 타입

### 1. **TIMESTAMPTZ (추천)**
```sql
-- UTC 시간대 정보가 포함된 타임스탬프 (권장)
created_at TIMESTAMPTZ NOT NULL,
modified_at TIMESTAMPTZ NOT NULL,
deleted_at TIMESTAMPTZ NULL
```

**장점:**
- ISO8601 문자열을 자동으로 파싱
- 시간대 정보 보존 (현재 데이터의 'Z'는 UTC를 의미)
- 시간대 변환 자동 지원
- 인덱싱 및 쿼리 성능 최적화

**예시:**
```sql
-- 데이터 삽입 시
INSERT INTO recipes (created_at) VALUES ('2025-01-18T13:13:36Z');
-- 또는
INSERT INTO recipes (created_at) VALUES (NOW());
```

### 2. **TIMESTAMP (시간대 정보 불필요시)**
```sql
-- 시간대 정보 없는 타임스탬프
created_at TIMESTAMP NOT NULL,
modified_at TIMESTAMP NOT NULL
```

**사용 케이스:**
- 모든 데이터가 동일한 시간대인 경우
- 시간대 변환이 불필요한 경우

### 3. **TEXT/VARCHAR (권장하지 않음)**
```sql
-- 문자열로 저장 (권장하지 않음)
created_at VARCHAR(25)
```

**단점:**
- 날짜/시간 연산 불가
- 정렬 및 비교 연산의 성능 저하
- 데이터 무결성 보장 어려움

## 테이블 생성 DDL 예시

### Recipe 테이블
```sql
CREATE TABLE recipes (
    id BIGSERIAL PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    name VARCHAR(100),
    user_name VARCHAR(50),
    user_nickname VARCHAR(50),
    view_cnt BIGINT DEFAULT 0,
    recommend_cnt BIGINT DEFAULT 0,
    scrap_cnt BIGINT DEFAULT 0,
    method_type VARCHAR(20),
    situation_type VARCHAR(20),
    ingredient_type VARCHAR(30),
    dish_type VARCHAR(30),
    content TEXT,
    ingredient_list TEXT,
    serving VARCHAR(20),
    difficulty VARCHAR(20),
    cooking_time VARCHAR(30),
    original_created_at BIGINT,
    image_url TEXT,
    user_id BIGINT REFERENCES users(id),
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    created_by BIGINT NOT NULL,
    modified_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    modified_by BIGINT NOT NULL,
    is_deleted BOOLEAN DEFAULT FALSE
);
```

### User 테이블
```sql
CREATE TABLE users (
    id BIGSERIAL PRIMARY KEY,
    nickname VARCHAR(50) UNIQUE NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    password VARCHAR(100) NOT NULL,
    sex_enum VARCHAR(10) CHECK (sex_enum IN ('MALE', 'FEMALE')),
    age_enum VARCHAR(15) CHECK (age_enum IN ('TWENTIES', 'THIRTIES', 'FORTY_PLUS')),
    role_enum VARCHAR(20) DEFAULT 'USER',
    created_by BIGINT NOT NULL,
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    modified_by BIGINT NOT NULL,
    modified_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    deleted_by BIGINT NULL,
    deleted_at TIMESTAMPTZ NULL,
    is_deleted BOOLEAN DEFAULT FALSE
);
```

### User Profiles 테이블
```sql
CREATE TABLE user_profiles (
    user_id BIGINT PRIMARY KEY REFERENCES users(id),
    demographic_segment VARCHAR(20),
    engagement_level VARCHAR(10),
    cooking_style_preference VARCHAR(50),
    segment_calculated_at TIMESTAMPTZ,
    behavior_calculated_at TIMESTAMPTZ,
    created_by BIGINT NOT NULL,
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    modified_by BIGINT NOT NULL,
    modified_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    deleted_by BIGINT NULL,
    deleted_at TIMESTAMPTZ NULL
);
```

## 데이터 삽입 시 주의사항

### Python에서 pandas를 사용한 삽입
```python
# ISO8601 문자열을 그대로 사용 가능
df.to_sql('recipes', engine, if_exists='append', index=False, 
          dtype={'created_at': 'TIMESTAMPTZ', 'modified_at': 'TIMESTAMPTZ'})
```

### psycopg2 사용 시
```python
# 자동 변환됨
cursor.execute(
    "INSERT INTO recipes (created_at) VALUES (%s)",
    ('2025-01-18T13:13:36Z',)
)
```

## 결론
현재 데이터의 ISO8601 형태("2025-01-18T13:13:36Z")를 PostgreSQL에 저장할 때는 **TIMESTAMPTZ** 타입을 사용하는 것을 강력히 권장합니다.